This Colab notebook demonstrates how to do a **patent classification** task (CPC subclass level) based on the fine-tuned BERT model (**PatentBERT**). 

This notebook will download related model files, the CPC label file, and create required source code automatically. If you'd like to test with your own data, you may update the "**test_data.tsv**" content in this notebook. 

**Citation for this work:**
[https://doi.org/10.1016/j.wpi.2020.101965](https://doi.org/10.1016/j.wpi.2020.101965) 

```
@article{patent_bert, 
  author = "Jieh-Sheng Lee and Jieh Hsiang",
  title = "{PatentBERT: Patent classification with fine-tuning a pre-trained BERT model}",
  journal = "World Patent Information",
  volume = "61",
  number = "101965",
  year = "2020",
}
```

In [ ]:
%tensorflow_version 1.x
import os
import time
import tensorflow as tf

os.environ['TZ'] = 'Asia/Taipei'
time.tzset()
print(time.strftime('%X %x %Z'))
print('tf version: %s' % tf.__version__)

TensorFlow 1.x selected.
12:52:22 10/02/20 CST
tf version: 1.15.2


In [ ]:
# download fine-tuned model files, CPC lable file, data.2015.tsv

!pip -q install gdown

model_folder = '/content/PatentBERT'
if os.path.exists(model_folder) == False:
  !mkdir $model_folder; cd $model_folder; \
  gdown https://drive.google.com/uc?id=1xXGUhdcUWqlqBuL-1GqdCjUysvKGxbaY; \
  gdown https://drive.google.com/uc?id=1-66JQrSBDyr4ohFatxXuwH1NpXJEBvRH; \
  gdown https://drive.google.com/uc?id=1-5JK4KihQwZhvXZC5Y6W15abI9w-jl6E; \
  gdown https://drive.google.com/uc?id=1lnqq-JDNZJ0oixLoNAZGoL5NqU-r6EPL; \
  gdown https://drive.google.com/uc?id=1fVGPfhigflseut5QleXp1ma9imHYZkOF

!ls $model_folder

device_name = tf.test.gpu_device_name()
if 'GPU' in device_name:
  print('GPU ready: %s' % device_name) 
  GPU_FLAG = True
else:
  print('CPU only.....')    

Downloading...
From: https://drive.google.com/uc?id=1xXGUhdcUWqlqBuL-1GqdCjUysvKGxbaY
To: /content/PatentBERT/model.ckpt-181172.data-00000-of-00001
1.32GB [00:05, 245MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-66JQrSBDyr4ohFatxXuwH1NpXJEBvRH
To: /content/PatentBERT/model.ckpt-181172.index
100% 9.09k/9.09k [00:00<00:00, 7.83MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5JK4KihQwZhvXZC5Y6W15abI9w-jl6E
To: /content/PatentBERT/model.ckpt-181172.meta
4.85MB [00:00, 67.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lnqq-JDNZJ0oixLoNAZGoL5NqU-r6EPL
To: /content/PatentBERT/data.2015.tsv
370MB [00:01, 195MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fVGPfhigflseut5QleXp1ma9imHYZkOF
To: /content/PatentBERT/labels_group_id.tsv
100% 58.4k/58.4k [00:00<00:00, 16.6MB/s]
data.2015.tsv			       model.ckpt-181172.index
labels_group_id.tsv		       model.ckpt-181172.meta
model.ckpt-181172.data-00000-of-00001
GPU ready: /device:GPU:0


In [ ]:
%%writefile optimization.py
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Functions and classes related to optimization (weight updates)."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re
import tensorflow as tf


def create_optimizer(loss, init_lr, num_train_steps, num_warmup_steps, use_tpu):
  """Creates an optimizer training op."""
  global_step = tf.train.get_or_create_global_step()

  learning_rate = tf.constant(value=init_lr, shape=[], dtype=tf.float32)

  # Implements linear decay of the learning rate.
  learning_rate = tf.train.polynomial_decay(
      learning_rate,
      global_step,
      num_train_steps,
      end_learning_rate=0.0,
      power=1.0,
      cycle=False)

  # Implements linear warmup. I.e., if global_step < num_warmup_steps, the
  # learning rate will be `global_step/num_warmup_steps * init_lr`.
  if num_warmup_steps:
    global_steps_int = tf.cast(global_step, tf.int32)
    warmup_steps_int = tf.constant(num_warmup_steps, dtype=tf.int32)

    global_steps_float = tf.cast(global_steps_int, tf.float32)
    warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

    warmup_percent_done = global_steps_float / warmup_steps_float
    warmup_learning_rate = init_lr * warmup_percent_done

    is_warmup = tf.cast(global_steps_int < warmup_steps_int, tf.float32)
    learning_rate = (
        (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)

  # It is recommended that you use this optimizer for fine tuning, since this
  # is how the model was trained (note that the Adam m/v variables are NOT
  # loaded from init_checkpoint.)
  optimizer = AdamWeightDecayOptimizer(
      learning_rate=learning_rate,
      weight_decay_rate=0.01,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-6,
      exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"])

  if use_tpu:
    optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

  tvars = tf.trainable_variables()
  grads = tf.gradients(loss, tvars)

  # This is how the model was pre-trained.
  (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)

  train_op = optimizer.apply_gradients(
      zip(grads, tvars), global_step=global_step)

  # Normally the global step update is done inside of `apply_gradients`.
  # However, `AdamWeightDecayOptimizer` doesn't do this. But if you use
  # a different optimizer, you should probably take this line out.
  new_global_step = global_step + 1
  train_op = tf.group(train_op, [global_step.assign(new_global_step)])
  return train_op


class AdamWeightDecayOptimizer(tf.train.Optimizer):
  """A basic Adam optimizer that includes "correct" L2 weight decay."""

  def __init__(self,
               learning_rate,
               weight_decay_rate=0.0,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=1e-6,
               exclude_from_weight_decay=None,
               name="AdamWeightDecayOptimizer"):
    """Constructs a AdamWeightDecayOptimizer."""
    super(AdamWeightDecayOptimizer, self).__init__(False, name)

    self.learning_rate = learning_rate
    self.weight_decay_rate = weight_decay_rate
    self.beta_1 = beta_1
    self.beta_2 = beta_2
    self.epsilon = epsilon
    self.exclude_from_weight_decay = exclude_from_weight_decay

  def apply_gradients(self, grads_and_vars, global_step=None, name=None):
    """See base class."""
    assignments = []
    for (grad, param) in grads_and_vars:
      if grad is None or param is None:
        continue

      param_name = self._get_variable_name(param.name)

      m = tf.get_variable(
          name=param_name + "/adam_m",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())
      v = tf.get_variable(
          name=param_name + "/adam_v",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())

      # Standard Adam update.
      next_m = (
          tf.multiply(self.beta_1, m) + tf.multiply(1.0 - self.beta_1, grad))
      next_v = (
          tf.multiply(self.beta_2, v) + tf.multiply(1.0 - self.beta_2,
                                                    tf.square(grad)))

      update = next_m / (tf.sqrt(next_v) + self.epsilon)

      # Just adding the square of the weights to the loss function is *not*
      # the correct way of using L2 regularization/weight decay with Adam,
      # since that will interact with the m and v parameters in strange ways.
      #
      # Instead we want ot decay the weights in a manner that doesn't interact
      # with the m/v parameters. This is equivalent to adding the square
      # of the weights to the loss with plain (non-momentum) SGD.
      if self._do_use_weight_decay(param_name):
        update += self.weight_decay_rate * param

      update_with_lr = self.learning_rate * update

      next_param = param - update_with_lr

      assignments.extend(
          [param.assign(next_param),
           m.assign(next_m),
           v.assign(next_v)])
    return tf.group(*assignments, name=name)

  def _do_use_weight_decay(self, param_name):
    """Whether to use L2 weight decay for `param_name`."""
    if not self.weight_decay_rate:
      return False
    if self.exclude_from_weight_decay:
      for r in self.exclude_from_weight_decay:
        if re.search(r, param_name) is not None:
          return False
    return True

  def _get_variable_name(self, param_name):
    """Get the variable name from the tensor name."""
    m = re.match("^(.*):\\d+$", param_name)
    if m is not None:
      param_name = m.group(1)
    return param_name


Writing optimization.py


In [ ]:
%%writefile modeling.py
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""The main BERT model and related functions."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import copy
import json
import math
import re
import six
import tensorflow as tf


class BertConfig(object):
  """Configuration for `BertModel`."""

  def __init__(self,
               vocab_size,
               hidden_size=768,
               num_hidden_layers=12,
               num_attention_heads=12,
               intermediate_size=3072,
               hidden_act="gelu",
               hidden_dropout_prob=0.1,
               attention_probs_dropout_prob=0.1,
               max_position_embeddings=512,
               type_vocab_size=16,
               initializer_range=0.02):
    """Constructs BertConfig.

    Args:
      vocab_size: Vocabulary size of `inputs_ids` in `BertModel`.
      hidden_size: Size of the encoder layers and the pooler layer.
      num_hidden_layers: Number of hidden layers in the Transformer encoder.
      num_attention_heads: Number of attention heads for each attention layer in
        the Transformer encoder.
      intermediate_size: The size of the "intermediate" (i.e., feed-forward)
        layer in the Transformer encoder.
      hidden_act: The non-linear activation function (function or string) in the
        encoder and pooler.
      hidden_dropout_prob: The dropout probability for all fully connected
        layers in the embeddings, encoder, and pooler.
      attention_probs_dropout_prob: The dropout ratio for the attention
        probabilities.
      max_position_embeddings: The maximum sequence length that this model might
        ever be used with. Typically set this to something large just in case
        (e.g., 512 or 1024 or 2048).
      type_vocab_size: The vocabulary size of the `token_type_ids` passed into
        `BertModel`.
      initializer_range: The stdev of the truncated_normal_initializer for
        initializing all weight matrices.
    """
    self.vocab_size = vocab_size
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.num_attention_heads = num_attention_heads
    self.hidden_act = hidden_act
    self.intermediate_size = intermediate_size
    self.hidden_dropout_prob = hidden_dropout_prob
    self.attention_probs_dropout_prob = attention_probs_dropout_prob
    self.max_position_embeddings = max_position_embeddings
    self.type_vocab_size = type_vocab_size
    self.initializer_range = initializer_range

  @classmethod
  def from_dict(cls, json_object):
    """Constructs a `BertConfig` from a Python dictionary of parameters."""
    config = BertConfig(vocab_size=None)
    for (key, value) in six.iteritems(json_object):
      config.__dict__[key] = value
    return config

  @classmethod
  def from_json_file(cls, json_file):
    """Constructs a `BertConfig` from a json file of parameters."""
    with tf.gfile.GFile(json_file, "r") as reader:
      text = reader.read()
    return cls.from_dict(json.loads(text))

  def to_dict(self):
    """Serializes this instance to a Python dictionary."""
    output = copy.deepcopy(self.__dict__)
    return output

  def to_json_string(self):
    """Serializes this instance to a JSON string."""
    return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


class BertModel(object):
  """BERT model ("Bidirectional Encoder Representations from Transformers").

  Example usage:

  ```python
  # Already been converted into WordPiece token ids
  input_ids = tf.constant([[31, 51, 99], [15, 5, 0]])
  input_mask = tf.constant([[1, 1, 1], [1, 1, 0]])
  token_type_ids = tf.constant([[0, 0, 1], [0, 2, 0]])

  config = modeling.BertConfig(vocab_size=32000, hidden_size=512,
    num_hidden_layers=8, num_attention_heads=6, intermediate_size=1024)

  model = modeling.BertModel(config=config, is_training=True,
    input_ids=input_ids, input_mask=input_mask, token_type_ids=token_type_ids)

  label_embeddings = tf.get_variable(...)
  pooled_output = model.get_pooled_output()
  logits = tf.matmul(pooled_output, label_embeddings)
  ...
  ```
  """

  def __init__(self,
               config,
               is_training,
               input_ids,
               input_mask=None,
               token_type_ids=None,
               use_one_hot_embeddings=True,
               scope=None):
    """Constructor for BertModel.

    Args:
      config: `BertConfig` instance.
      is_training: bool. true for training model, false for eval model. Controls
        whether dropout will be applied.
      input_ids: int32 Tensor of shape [batch_size, seq_length].
      input_mask: (optional) int32 Tensor of shape [batch_size, seq_length].
      token_type_ids: (optional) int32 Tensor of shape [batch_size, seq_length].
      use_one_hot_embeddings: (optional) bool. Whether to use one-hot word
        embeddings or tf.embedding_lookup() for the word embeddings. On the TPU,
        it is much faster if this is True, on the CPU or GPU, it is faster if
        this is False.
      scope: (optional) variable scope. Defaults to "bert".

    Raises:
      ValueError: The config is invalid or one of the input tensor shapes
        is invalid.
    """
    config = copy.deepcopy(config)
    if not is_training:
      config.hidden_dropout_prob = 0.0
      config.attention_probs_dropout_prob = 0.0

    input_shape = get_shape_list(input_ids, expected_rank=2)
    batch_size = input_shape[0]
    seq_length = input_shape[1]

    if input_mask is None:
      input_mask = tf.ones(shape=[batch_size, seq_length], dtype=tf.int32)

    if token_type_ids is None:
      token_type_ids = tf.zeros(shape=[batch_size, seq_length], dtype=tf.int32)

    with tf.variable_scope(scope, default_name="bert"):
      with tf.variable_scope("embeddings"):
        # Perform embedding lookup on the word ids.
        (self.embedding_output, self.embedding_table) = embedding_lookup(
            input_ids=input_ids,
            vocab_size=config.vocab_size,
            embedding_size=config.hidden_size,
            initializer_range=config.initializer_range,
            word_embedding_name="word_embeddings",
            use_one_hot_embeddings=use_one_hot_embeddings)

        # Add positional embeddings and token type embeddings, then layer
        # normalize and perform dropout.
        self.embedding_output = embedding_postprocessor(
            input_tensor=self.embedding_output,
            use_token_type=True,
            token_type_ids=token_type_ids,
            token_type_vocab_size=config.type_vocab_size,
            token_type_embedding_name="token_type_embeddings",
            use_position_embeddings=True,
            position_embedding_name="position_embeddings",
            initializer_range=config.initializer_range,
            max_position_embeddings=config.max_position_embeddings,
            dropout_prob=config.hidden_dropout_prob)

      with tf.variable_scope("encoder"):
        # This converts a 2D mask of shape [batch_size, seq_length] to a 3D
        # mask of shape [batch_size, seq_length, seq_length] which is used
        # for the attention scores.
        attention_mask = create_attention_mask_from_input_mask(
            input_ids, input_mask)

        # Run the stacked transformer.
        # `sequence_output` shape = [batch_size, seq_length, hidden_size].
        self.all_encoder_layers = transformer_model(
            input_tensor=self.embedding_output,
            attention_mask=attention_mask,
            hidden_size=config.hidden_size,
            num_hidden_layers=config.num_hidden_layers,
            num_attention_heads=config.num_attention_heads,
            intermediate_size=config.intermediate_size,
            intermediate_act_fn=get_activation(config.hidden_act),
            hidden_dropout_prob=config.hidden_dropout_prob,
            attention_probs_dropout_prob=config.attention_probs_dropout_prob,
            initializer_range=config.initializer_range,
            do_return_all_layers=True)

      self.sequence_output = self.all_encoder_layers[-1]
      # The "pooler" converts the encoded sequence tensor of shape
      # [batch_size, seq_length, hidden_size] to a tensor of shape
      # [batch_size, hidden_size]. This is necessary for segment-level
      # (or segment-pair-level) classification tasks where we need a fixed
      # dimensional representation of the segment.
      with tf.variable_scope("pooler"):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token. We assume that this has been pre-trained
        first_token_tensor = tf.squeeze(self.sequence_output[:, 0:1, :], axis=1)
        self.pooled_output = tf.layers.dense(
            first_token_tensor,
            config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer(config.initializer_range))

  def get_pooled_output(self):
    return self.pooled_output

  def get_sequence_output(self):
    """Gets final hidden layer of encoder.

    Returns:
      float Tensor of shape [batch_size, seq_length, hidden_size] corresponding
      to the final hidden of the transformer encoder.
    """
    return self.sequence_output

  def get_all_encoder_layers(self):
    return self.all_encoder_layers

  def get_embedding_output(self):
    """Gets output of the embedding lookup (i.e., input to the transformer).

    Returns:
      float Tensor of shape [batch_size, seq_length, hidden_size] corresponding
      to the output of the embedding layer, after summing the word
      embeddings with the positional embeddings and the token type embeddings,
      then performing layer normalization. This is the input to the transformer.
    """
    return self.embedding_output

  def get_embedding_table(self):
    return self.embedding_table


def gelu(input_tensor):
  """Gaussian Error Linear Unit.

  This is a smoother version of the RELU.
  Original paper: https://arxiv.org/abs/1606.08415

  Args:
    input_tensor: float Tensor to perform activation.

  Returns:
    `input_tensor` with the GELU activation applied.
  """
  cdf = 0.5 * (1.0 + tf.erf(input_tensor / tf.sqrt(2.0)))
  return input_tensor * cdf


def get_activation(activation_string):
  """Maps a string to a Python function, e.g., "relu" => `tf.nn.relu`.

  Args:
    activation_string: String name of the activation function.

  Returns:
    A Python function corresponding to the activation function. If
    `activation_string` is None, empty, or "linear", this will return None.
    If `activation_string` is not a string, it will return `activation_string`.

  Raises:
    ValueError: The `activation_string` does not correspond to a known
      activation.
  """

  # We assume that anything that"s not a string is already an activation
  # function, so we just return it.
  if not isinstance(activation_string, six.string_types):
    return activation_string

  if not activation_string:
    return None

  act = activation_string.lower()
  if act == "linear":
    return None
  elif act == "relu":
    return tf.nn.relu
  elif act == "gelu":
    return gelu
  elif act == "tanh":
    return tf.tanh
  else:
    raise ValueError("Unsupported activation: %s" % act)


def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
  """Compute the union of the current variables and checkpoint variables."""
  assignment_map = {}
  initialized_variable_names = {}

  name_to_variable = collections.OrderedDict()
  for var in tvars:
    name = var.name
    m = re.match("^(.*):\\d+$", name)
    if m is not None:
      name = m.group(1)
    name_to_variable[name] = var

  init_vars = tf.train.list_variables(init_checkpoint)

  assignment_map = collections.OrderedDict()
  for x in init_vars:
    (name, var) = (x[0], x[1])
    if name not in name_to_variable:
      continue
    assignment_map[name] = name
    initialized_variable_names[name] = 1
    initialized_variable_names[name + ":0"] = 1

  return (assignment_map, initialized_variable_names)


def dropout(input_tensor, dropout_prob):
  """Perform dropout.

  Args:
    input_tensor: float Tensor.
    dropout_prob: Python float. The probability of dropping out a value (NOT of
      *keeping* a dimension as in `tf.nn.dropout`).

  Returns:
    A version of `input_tensor` with dropout applied.
  """
  if dropout_prob is None or dropout_prob == 0.0:
    return input_tensor

  output = tf.nn.dropout(input_tensor, 1.0 - dropout_prob)
  return output


def layer_norm(input_tensor, name=None):
  """Run layer normalization on the last dimension of the tensor."""
  return tf.contrib.layers.layer_norm(
      inputs=input_tensor, begin_norm_axis=-1, begin_params_axis=-1, scope=name)


def layer_norm_and_dropout(input_tensor, dropout_prob, name=None):
  """Runs layer normalization followed by dropout."""
  output_tensor = layer_norm(input_tensor, name)
  output_tensor = dropout(output_tensor, dropout_prob)
  return output_tensor


def create_initializer(initializer_range=0.02):
  """Creates a `truncated_normal_initializer` with the given range."""
  return tf.truncated_normal_initializer(stddev=initializer_range)


def embedding_lookup(input_ids,
                     vocab_size,
                     embedding_size=128,
                     initializer_range=0.02,
                     word_embedding_name="word_embeddings",
                     use_one_hot_embeddings=False):
  """Looks up words embeddings for id tensor.

  Args:
    input_ids: int32 Tensor of shape [batch_size, seq_length] containing word
      ids.
    vocab_size: int. Size of the embedding vocabulary.
    embedding_size: int. Width of the word embeddings.
    initializer_range: float. Embedding initialization range.
    word_embedding_name: string. Name of the embedding table.
    use_one_hot_embeddings: bool. If True, use one-hot method for word
      embeddings. If False, use `tf.nn.embedding_lookup()`. One hot is better
      for TPUs.

  Returns:
    float Tensor of shape [batch_size, seq_length, embedding_size].
  """
  # This function assumes that the input is of shape [batch_size, seq_length,
  # num_inputs].
  #
  # If the input is a 2D tensor of shape [batch_size, seq_length], we
  # reshape to [batch_size, seq_length, 1].
  if input_ids.shape.ndims == 2:
    input_ids = tf.expand_dims(input_ids, axis=[-1])

  embedding_table = tf.get_variable(
      name=word_embedding_name,
      shape=[vocab_size, embedding_size],
      initializer=create_initializer(initializer_range))

  if use_one_hot_embeddings:
    flat_input_ids = tf.reshape(input_ids, [-1])
    one_hot_input_ids = tf.one_hot(flat_input_ids, depth=vocab_size)
    output = tf.matmul(one_hot_input_ids, embedding_table)
  else:
    output = tf.nn.embedding_lookup(embedding_table, input_ids)

  input_shape = get_shape_list(input_ids)

  output = tf.reshape(output,
                      input_shape[0:-1] + [input_shape[-1] * embedding_size])
  return (output, embedding_table)


def embedding_postprocessor(input_tensor,
                            use_token_type=False,
                            token_type_ids=None,
                            token_type_vocab_size=16,
                            token_type_embedding_name="token_type_embeddings",
                            use_position_embeddings=True,
                            position_embedding_name="position_embeddings",
                            initializer_range=0.02,
                            max_position_embeddings=512,
                            dropout_prob=0.1):
  """Performs various post-processing on a word embedding tensor.

  Args:
    input_tensor: float Tensor of shape [batch_size, seq_length,
      embedding_size].
    use_token_type: bool. Whether to add embeddings for `token_type_ids`.
    token_type_ids: (optional) int32 Tensor of shape [batch_size, seq_length].
      Must be specified if `use_token_type` is True.
    token_type_vocab_size: int. The vocabulary size of `token_type_ids`.
    token_type_embedding_name: string. The name of the embedding table variable
      for token type ids.
    use_position_embeddings: bool. Whether to add position embeddings for the
      position of each token in the sequence.
    position_embedding_name: string. The name of the embedding table variable
      for positional embeddings.
    initializer_range: float. Range of the weight initialization.
    max_position_embeddings: int. Maximum sequence length that might ever be
      used with this model. This can be longer than the sequence length of
      input_tensor, but cannot be shorter.
    dropout_prob: float. Dropout probability applied to the final output tensor.

  Returns:
    float tensor with same shape as `input_tensor`.

  Raises:
    ValueError: One of the tensor shapes or input values is invalid.
  """
  input_shape = get_shape_list(input_tensor, expected_rank=3)
  batch_size = input_shape[0]
  seq_length = input_shape[1]
  width = input_shape[2]

  output = input_tensor

  if use_token_type:
    if token_type_ids is None:
      raise ValueError("`token_type_ids` must be specified if"
                       "`use_token_type` is True.")
    token_type_table = tf.get_variable(
        name=token_type_embedding_name,
        shape=[token_type_vocab_size, width],
        initializer=create_initializer(initializer_range))
    # This vocab will be small so we always do one-hot here, since it is always
    # faster for a small vocabulary.
    flat_token_type_ids = tf.reshape(token_type_ids, [-1])
    one_hot_ids = tf.one_hot(flat_token_type_ids, depth=token_type_vocab_size)
    token_type_embeddings = tf.matmul(one_hot_ids, token_type_table)
    token_type_embeddings = tf.reshape(token_type_embeddings,
                                       [batch_size, seq_length, width])
    output += token_type_embeddings

  if use_position_embeddings:
    assert_op = tf.assert_less_equal(seq_length, max_position_embeddings)
    with tf.control_dependencies([assert_op]):
      full_position_embeddings = tf.get_variable(
          name=position_embedding_name,
          shape=[max_position_embeddings, width],
          initializer=create_initializer(initializer_range))
      # Since the position embedding table is a learned variable, we create it
      # using a (long) sequence length `max_position_embeddings`. The actual
      # sequence length might be shorter than this, for faster training of
      # tasks that do not have long sequences.
      #
      # So `full_position_embeddings` is effectively an embedding table
      # for position [0, 1, 2, ..., max_position_embeddings-1], and the current
      # sequence has positions [0, 1, 2, ... seq_length-1], so we can just
      # perform a slice.
      position_embeddings = tf.slice(full_position_embeddings, [0, 0],
                                     [seq_length, -1])
      num_dims = len(output.shape.as_list())

      # Only the last two dimensions are relevant (`seq_length` and `width`), so
      # we broadcast among the first dimensions, which is typically just
      # the batch size.
      position_broadcast_shape = []
      for _ in range(num_dims - 2):
        position_broadcast_shape.append(1)
      position_broadcast_shape.extend([seq_length, width])
      position_embeddings = tf.reshape(position_embeddings,
                                       position_broadcast_shape)
      output += position_embeddings

  output = layer_norm_and_dropout(output, dropout_prob)
  return output


def create_attention_mask_from_input_mask(from_tensor, to_mask):
  """Create 3D attention mask from a 2D tensor mask.

  Args:
    from_tensor: 2D or 3D Tensor of shape [batch_size, from_seq_length, ...].
    to_mask: int32 Tensor of shape [batch_size, to_seq_length].

  Returns:
    float Tensor of shape [batch_size, from_seq_length, to_seq_length].
  """
  from_shape = get_shape_list(from_tensor, expected_rank=[2, 3])
  batch_size = from_shape[0]
  from_seq_length = from_shape[1]

  to_shape = get_shape_list(to_mask, expected_rank=2)
  to_seq_length = to_shape[1]

  to_mask = tf.cast(
      tf.reshape(to_mask, [batch_size, 1, to_seq_length]), tf.float32)

  # We don't assume that `from_tensor` is a mask (although it could be). We
  # don't actually care if we attend *from* padding tokens (only *to* padding)
  # tokens so we create a tensor of all ones.
  #
  # `broadcast_ones` = [batch_size, from_seq_length, 1]
  broadcast_ones = tf.ones(
      shape=[batch_size, from_seq_length, 1], dtype=tf.float32)

  # Here we broadcast along two dimensions to create the mask.
  mask = broadcast_ones * to_mask

  return mask


def attention_layer(from_tensor,
                    to_tensor,
                    attention_mask=None,
                    num_attention_heads=1,
                    size_per_head=512,
                    query_act=None,
                    key_act=None,
                    value_act=None,
                    attention_probs_dropout_prob=0.0,
                    initializer_range=0.02,
                    do_return_2d_tensor=False,
                    batch_size=None,
                    from_seq_length=None,
                    to_seq_length=None):
  """Performs multi-headed attention from `from_tensor` to `to_tensor`.

  This is an implementation of multi-headed attention based on "Attention
  is all you Need". If `from_tensor` and `to_tensor` are the same, then
  this is self-attention. Each timestep in `from_tensor` attends to the
  corresponding sequence in `to_tensor`, and returns a fixed-with vector.

  This function first projects `from_tensor` into a "query" tensor and
  `to_tensor` into "key" and "value" tensors. These are (effectively) a list
  of tensors of length `num_attention_heads`, where each tensor is of shape
  [batch_size, seq_length, size_per_head].

  Then, the query and key tensors are dot-producted and scaled. These are
  softmaxed to obtain attention probabilities. The value tensors are then
  interpolated by these probabilities, then concatenated back to a single
  tensor and returned.

  In practice, the multi-headed attention are done with transposes and
  reshapes rather than actual separate tensors.

  Args:
    from_tensor: float Tensor of shape [batch_size, from_seq_length,
      from_width].
    to_tensor: float Tensor of shape [batch_size, to_seq_length, to_width].
    attention_mask: (optional) int32 Tensor of shape [batch_size,
      from_seq_length, to_seq_length]. The values should be 1 or 0. The
      attention scores will effectively be set to -infinity for any positions in
      the mask that are 0, and will be unchanged for positions that are 1.
    num_attention_heads: int. Number of attention heads.
    size_per_head: int. Size of each attention head.
    query_act: (optional) Activation function for the query transform.
    key_act: (optional) Activation function for the key transform.
    value_act: (optional) Activation function for the value transform.
    attention_probs_dropout_prob: (optional) float. Dropout probability of the
      attention probabilities.
    initializer_range: float. Range of the weight initializer.
    do_return_2d_tensor: bool. If True, the output will be of shape [batch_size
      * from_seq_length, num_attention_heads * size_per_head]. If False, the
      output will be of shape [batch_size, from_seq_length, num_attention_heads
      * size_per_head].
    batch_size: (Optional) int. If the input is 2D, this might be the batch size
      of the 3D version of the `from_tensor` and `to_tensor`.
    from_seq_length: (Optional) If the input is 2D, this might be the seq length
      of the 3D version of the `from_tensor`.
    to_seq_length: (Optional) If the input is 2D, this might be the seq length
      of the 3D version of the `to_tensor`.

  Returns:
    float Tensor of shape [batch_size, from_seq_length,
      num_attention_heads * size_per_head]. (If `do_return_2d_tensor` is
      true, this will be of shape [batch_size * from_seq_length,
      num_attention_heads * size_per_head]).

  Raises:
    ValueError: Any of the arguments or tensor shapes are invalid.
  """

  def transpose_for_scores(input_tensor, batch_size, num_attention_heads,
                           seq_length, width):
    output_tensor = tf.reshape(
        input_tensor, [batch_size, seq_length, num_attention_heads, width])

    output_tensor = tf.transpose(output_tensor, [0, 2, 1, 3])
    return output_tensor

  from_shape = get_shape_list(from_tensor, expected_rank=[2, 3])
  to_shape = get_shape_list(to_tensor, expected_rank=[2, 3])

  if len(from_shape) != len(to_shape):
    raise ValueError(
        "The rank of `from_tensor` must match the rank of `to_tensor`.")

  if len(from_shape) == 3:
    batch_size = from_shape[0]
    from_seq_length = from_shape[1]
    to_seq_length = to_shape[1]
  elif len(from_shape) == 2:
    if (batch_size is None or from_seq_length is None or to_seq_length is None):
      raise ValueError(
          "When passing in rank 2 tensors to attention_layer, the values "
          "for `batch_size`, `from_seq_length`, and `to_seq_length` "
          "must all be specified.")

  # Scalar dimensions referenced here:
  #   B = batch size (number of sequences)
  #   F = `from_tensor` sequence length
  #   T = `to_tensor` sequence length
  #   N = `num_attention_heads`
  #   H = `size_per_head`

  from_tensor_2d = reshape_to_matrix(from_tensor)
  to_tensor_2d = reshape_to_matrix(to_tensor)

  # `query_layer` = [B*F, N*H]
  query_layer = tf.layers.dense(
      from_tensor_2d,
      num_attention_heads * size_per_head,
      activation=query_act,
      name="query",
      kernel_initializer=create_initializer(initializer_range))

  # `key_layer` = [B*T, N*H]
  key_layer = tf.layers.dense(
      to_tensor_2d,
      num_attention_heads * size_per_head,
      activation=key_act,
      name="key",
      kernel_initializer=create_initializer(initializer_range))

  # `value_layer` = [B*T, N*H]
  value_layer = tf.layers.dense(
      to_tensor_2d,
      num_attention_heads * size_per_head,
      activation=value_act,
      name="value",
      kernel_initializer=create_initializer(initializer_range))

  # `query_layer` = [B, N, F, H]
  query_layer = transpose_for_scores(query_layer, batch_size,
                                     num_attention_heads, from_seq_length,
                                     size_per_head)

  # `key_layer` = [B, N, T, H]
  key_layer = transpose_for_scores(key_layer, batch_size, num_attention_heads,
                                   to_seq_length, size_per_head)

  # Take the dot product between "query" and "key" to get the raw
  # attention scores.
  # `attention_scores` = [B, N, F, T]
  attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
  attention_scores = tf.multiply(attention_scores,
                                 1.0 / math.sqrt(float(size_per_head)))

  if attention_mask is not None:
    # `attention_mask` = [B, 1, F, T]
    attention_mask = tf.expand_dims(attention_mask, axis=[1])

    # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
    # masked positions, this operation will create a tensor which is 0.0 for
    # positions we want to attend and -10000.0 for masked positions.
    adder = (1.0 - tf.cast(attention_mask, tf.float32)) * -10000.0

    # Since we are adding it to the raw scores before the softmax, this is
    # effectively the same as removing these entirely.
    attention_scores += adder

  # Normalize the attention scores to probabilities.
  # `attention_probs` = [B, N, F, T]
  attention_probs = tf.nn.softmax(attention_scores)

  # This is actually dropping out entire tokens to attend to, which might
  # seem a bit unusual, but is taken from the original Transformer paper.
  attention_probs = dropout(attention_probs, attention_probs_dropout_prob)

  # `value_layer` = [B, T, N, H]
  value_layer = tf.reshape(
      value_layer,
      [batch_size, to_seq_length, num_attention_heads, size_per_head])

  # `value_layer` = [B, N, T, H]
  value_layer = tf.transpose(value_layer, [0, 2, 1, 3])

  # `context_layer` = [B, N, F, H]
  context_layer = tf.matmul(attention_probs, value_layer)

  # `context_layer` = [B, F, N, H]
  context_layer = tf.transpose(context_layer, [0, 2, 1, 3])

  if do_return_2d_tensor:
    # `context_layer` = [B*F, N*H]
    context_layer = tf.reshape(
        context_layer,
        [batch_size * from_seq_length, num_attention_heads * size_per_head])
  else:
    # `context_layer` = [B, F, N*H]
    context_layer = tf.reshape(
        context_layer,
        [batch_size, from_seq_length, num_attention_heads * size_per_head])

  return context_layer


def transformer_model(input_tensor,
                      attention_mask=None,
                      hidden_size=768,
                      num_hidden_layers=12,
                      num_attention_heads=12,
                      intermediate_size=3072,
                      intermediate_act_fn=gelu,
                      hidden_dropout_prob=0.1,
                      attention_probs_dropout_prob=0.1,
                      initializer_range=0.02,
                      do_return_all_layers=False):
  """Multi-headed, multi-layer Transformer from "Attention is All You Need".

  This is almost an exact implementation of the original Transformer encoder.

  See the original paper:
  https://arxiv.org/abs/1706.03762

  Also see:
  https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py

  Args:
    input_tensor: float Tensor of shape [batch_size, seq_length, hidden_size].
    attention_mask: (optional) int32 Tensor of shape [batch_size, seq_length,
      seq_length], with 1 for positions that can be attended to and 0 in
      positions that should not be.
    hidden_size: int. Hidden size of the Transformer.
    num_hidden_layers: int. Number of layers (blocks) in the Transformer.
    num_attention_heads: int. Number of attention heads in the Transformer.
    intermediate_size: int. The size of the "intermediate" (a.k.a., feed
      forward) layer.
    intermediate_act_fn: function. The non-linear activation function to apply
      to the output of the intermediate/feed-forward layer.
    hidden_dropout_prob: float. Dropout probability for the hidden layers.
    attention_probs_dropout_prob: float. Dropout probability of the attention
      probabilities.
    initializer_range: float. Range of the initializer (stddev of truncated
      normal).
    do_return_all_layers: Whether to also return all layers or just the final
      layer.

  Returns:
    float Tensor of shape [batch_size, seq_length, hidden_size], the final
    hidden layer of the Transformer.

  Raises:
    ValueError: A Tensor shape or parameter is invalid.
  """
  if hidden_size % num_attention_heads != 0:
    raise ValueError(
        "The hidden size (%d) is not a multiple of the number of attention "
        "heads (%d)" % (hidden_size, num_attention_heads))

  attention_head_size = int(hidden_size / num_attention_heads)
  input_shape = get_shape_list(input_tensor, expected_rank=3)
  batch_size = input_shape[0]
  seq_length = input_shape[1]
  input_width = input_shape[2]

  # The Transformer performs sum residuals on all layers so the input needs
  # to be the same as the hidden size.
  if input_width != hidden_size:
    raise ValueError("The width of the input tensor (%d) != hidden size (%d)" %
                     (input_width, hidden_size))

  # We keep the representation as a 2D tensor to avoid re-shaping it back and
  # forth from a 3D tensor to a 2D tensor. Re-shapes are normally free on
  # the GPU/CPU but may not be free on the TPU, so we want to minimize them to
  # help the optimizer.
  prev_output = reshape_to_matrix(input_tensor)

  all_layer_outputs = []
  for layer_idx in range(num_hidden_layers):
    with tf.variable_scope("layer_%d" % layer_idx):
      layer_input = prev_output

      with tf.variable_scope("attention"):
        attention_heads = []
        with tf.variable_scope("self"):
          attention_head = attention_layer(
              from_tensor=layer_input,
              to_tensor=layer_input,
              attention_mask=attention_mask,
              num_attention_heads=num_attention_heads,
              size_per_head=attention_head_size,
              attention_probs_dropout_prob=attention_probs_dropout_prob,
              initializer_range=initializer_range,
              do_return_2d_tensor=True,
              batch_size=batch_size,
              from_seq_length=seq_length,
              to_seq_length=seq_length)
          attention_heads.append(attention_head)

        attention_output = None
        if len(attention_heads) == 1:
          attention_output = attention_heads[0]
        else:
          # In the case where we have other sequences, we just concatenate
          # them to the self-attention head before the projection.
          attention_output = tf.concat(attention_heads, axis=-1)

        # Run a linear projection of `hidden_size` then add a residual
        # with `layer_input`.
        with tf.variable_scope("output"):
          attention_output = tf.layers.dense(
              attention_output,
              hidden_size,
              kernel_initializer=create_initializer(initializer_range))
          attention_output = dropout(attention_output, hidden_dropout_prob)
          attention_output = layer_norm(attention_output + layer_input)

      # The activation is only applied to the "intermediate" hidden layer.
      with tf.variable_scope("intermediate"):
        intermediate_output = tf.layers.dense(
            attention_output,
            intermediate_size,
            activation=intermediate_act_fn,
            kernel_initializer=create_initializer(initializer_range))

      # Down-project back to `hidden_size` then add the residual.
      with tf.variable_scope("output"):
        layer_output = tf.layers.dense(
            intermediate_output,
            hidden_size,
            kernel_initializer=create_initializer(initializer_range))
        layer_output = dropout(layer_output, hidden_dropout_prob)
        layer_output = layer_norm(layer_output + attention_output)
        prev_output = layer_output
        all_layer_outputs.append(layer_output)

  if do_return_all_layers:
    final_outputs = []
    for layer_output in all_layer_outputs:
      final_output = reshape_from_matrix(layer_output, input_shape)
      final_outputs.append(final_output)
    return final_outputs
  else:
    final_output = reshape_from_matrix(prev_output, input_shape)
    return final_output


def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.

  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.

  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if name is None:
    name = tensor.name

  if expected_rank is not None:
    assert_rank(tensor, expected_rank, name)

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape


def reshape_to_matrix(input_tensor):
  """Reshapes a >= rank 2 tensor to a rank 2 tensor (i.e., a matrix)."""
  ndims = input_tensor.shape.ndims
  if ndims < 2:
    raise ValueError("Input tensor must have at least rank 2. Shape = %s" %
                     (input_tensor.shape))
  if ndims == 2:
    return input_tensor

  width = input_tensor.shape[-1]
  output_tensor = tf.reshape(input_tensor, [-1, width])
  return output_tensor


def reshape_from_matrix(output_tensor, orig_shape_list):
  """Reshapes a rank 2 tensor back to its original rank >= 2 tensor."""
  if len(orig_shape_list) == 2:
    return output_tensor

  output_shape = get_shape_list(output_tensor)

  orig_dims = orig_shape_list[0:-1]
  width = output_shape[-1]

  return tf.reshape(output_tensor, orig_dims + [width])


def assert_rank(tensor, expected_rank, name=None):
  """Raises an exception if the tensor rank is not of the expected rank.

  Args:
    tensor: A tf.Tensor to check the rank of.
    expected_rank: Python integer or list of integers, expected rank.
    name: Optional name of the tensor for the error message.

  Raises:
    ValueError: If the expected shape doesn't match the actual shape.
  """
  if name is None:
    name = tensor.name

  expected_rank_dict = {}
  if isinstance(expected_rank, six.integer_types):
    expected_rank_dict[expected_rank] = True
  else:
    for x in expected_rank:
      expected_rank_dict[x] = True

  actual_rank = tensor.shape.ndims
  if actual_rank not in expected_rank_dict:
    scope_name = tf.get_variable_scope().name
    raise ValueError(
        "For the tensor `%s` in scope `%s`, the actual rank "
        "`%d` (shape = %s) is not equal to the expected rank `%s`" %
        (name, scope_name, actual_rank, str(tensor.shape), str(expected_rank)))


Writing modeling.py


In [ ]:
%%writefile tokenization.py
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Tokenization classes."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import re
import unicodedata
import six
import tensorflow as tf


def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
  """Checks whether the casing config is consistent with the checkpoint name."""

  # The casing has to be passed in by the user and there is no explicit check
  # as to whether it matches the checkpoint. The casing information probably
  # should have been stored in the bert_config.json file, but it's not, so
  # we have to heuristically detect it to validate.

  if not init_checkpoint:
    return

  m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
  if m is None:
    return

  model_name = m.group(1)

  lower_models = [
      "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
      "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
  ]

  cased_models = [
      "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
      "multi_cased_L-12_H-768_A-12"
  ]

  is_bad_config = False
  if model_name in lower_models and not do_lower_case:
    is_bad_config = True
    actual_flag = "False"
    case_name = "lowercased"
    opposite_flag = "True"

  if model_name in cased_models and do_lower_case:
    is_bad_config = True
    actual_flag = "True"
    case_name = "cased"
    opposite_flag = "False"

  if is_bad_config:
    raise ValueError(
        "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
        "However, `%s` seems to be a %s model, so you "
        "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
        "how the model was pre-training. If this error is wrong, please "
        "just comment out this check." % (actual_flag, init_checkpoint,
                                          model_name, case_name, opposite_flag))


def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab = collections.OrderedDict()
  index = 0
  with tf.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = convert_to_unicode(reader.readline())
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1
  return vocab


def convert_by_vocab(vocab, items):
  """Converts a sequence of [tokens|ids] using the vocab."""
  output = []
  for item in items:
    output.append(vocab[item])
  return output


def convert_tokens_to_ids(vocab, tokens):
  return convert_by_vocab(vocab, tokens)


def convert_ids_to_tokens(inv_vocab, ids):
  return convert_by_vocab(inv_vocab, ids)


def whitespace_tokenize(text):
  """Runs basic whitespace cleaning and splitting on a piece of text."""
  text = text.strip()
  if not text:
    return []
  tokens = text.split()
  return tokens


class FullTokenizer(object):
  """Runs end-to-end tokenziation."""

  def __init__(self, vocab_file, do_lower_case=True):
    self.vocab = load_vocab(vocab_file)
    self.inv_vocab = {v: k for k, v in self.vocab.items()}
    self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    return split_tokens

  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)


class BasicTokenizer(object):
  """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

  def __init__(self, do_lower_case=True):
    """Constructs a BasicTokenizer.

    Args:
      do_lower_case: Whether to lower case the input.
    """
    self.do_lower_case = do_lower_case

  def tokenize(self, text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    text = self._clean_text(text)

    # This was added on November 1st, 2018 for the multilingual and Chinese
    # models. This is also applied to the English models now, but it doesn't
    # matter since the English models were not trained on any Chinese data
    # and generally don't have any Chinese data in them (there are Chinese
    # characters in the vocabulary because Wikipedia does have some Chinese
    # words in the English Wikipedia.).
    text = self._tokenize_chinese_chars(text)

    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
      if self.do_lower_case:
        token = token.lower()
        token = self._run_strip_accents(token)
      split_tokens.extend(self._run_split_on_punc(token))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens

  def _run_strip_accents(self, text):
    """Strips accents from a piece of text."""
    text = unicodedata.normalize("NFD", text)
    output = []
    for char in text:
      cat = unicodedata.category(char)
      if cat == "Mn":
        continue
      output.append(char)
    return "".join(output)

  def _run_split_on_punc(self, text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
      char = chars[i]
      if _is_punctuation(char):
        output.append([char])
        start_new_word = True
      else:
        if start_new_word:
          output.append([])
        start_new_word = False
        output[-1].append(char)
      i += 1

    return ["".join(x) for x in output]

  def _tokenize_chinese_chars(self, text):
    """Adds whitespace around any CJK character."""
    output = []
    for char in text:
      cp = ord(char)
      if self._is_chinese_char(cp):
        output.append(" ")
        output.append(char)
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)

  def _is_chinese_char(self, cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
        (cp >= 0x3400 and cp <= 0x4DBF) or  #
        (cp >= 0x20000 and cp <= 0x2A6DF) or  #
        (cp >= 0x2A700 and cp <= 0x2B73F) or  #
        (cp >= 0x2B740 and cp <= 0x2B81F) or  #
        (cp >= 0x2B820 and cp <= 0x2CEAF) or
        (cp >= 0xF900 and cp <= 0xFAFF) or  #
        (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
      return True

    return False

  def _clean_text(self, text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
      cp = ord(char)
      if cp == 0 or cp == 0xfffd or _is_control(char):
        continue
      if _is_whitespace(char):
        output.append(" ")
      else:
        output.append(char)
    return "".join(output)


class WordpieceTokenizer(object):
  """Runs WordPiece tokenziation."""

  def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=200):
    self.vocab = vocab
    self.unk_token = unk_token
    self.max_input_chars_per_word = max_input_chars_per_word

  def tokenize(self, text):
    """Tokenizes a piece of text into its word pieces.

    This uses a greedy longest-match-first algorithm to perform tokenization
    using the given vocabulary.

    For example:
      input = "unaffable"
      output = ["un", "##aff", "##able"]

    Args:
      text: A single token or whitespace separated tokens. This should have
        already been passed through `BasicTokenizer.

    Returns:
      A list of wordpiece tokens.
    """

    text = convert_to_unicode(text)

    output_tokens = []
    for token in whitespace_tokenize(text):
      chars = list(token)
      if len(chars) > self.max_input_chars_per_word:
        output_tokens.append(self.unk_token)
        continue

      is_bad = False
      start = 0
      sub_tokens = []
      while start < len(chars):
        end = len(chars)
        cur_substr = None
        while start < end:
          substr = "".join(chars[start:end])
          if start > 0:
            substr = "##" + substr
          if substr in self.vocab:
            cur_substr = substr
            break
          end -= 1
        if cur_substr is None:
          is_bad = True
          break
        sub_tokens.append(cur_substr)
        start = end

      if is_bad:
        output_tokens.append(self.unk_token)
      else:
        output_tokens.extend(sub_tokens)
    return output_tokens


def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically contorl characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  cat = unicodedata.category(char)
  if cat == "Zs":
    return True
  return False


def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat.startswith("C"):
    return True
  return False


def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False


Writing tokenization.py


In [ ]:
%%writefile patent_classifier.py
# coding=utf-8

# The following is based on https://github.com/google-research/bert

# Copyright 2020 Jason Lee
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""BERT finetuning runner."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pdb
import collections
import logging
import csv
import os
import modeling
import optimization
import tokenization
import numpy as np

from absl import flags

import absl.logging as _logging  # pylint: disable=unused-import
import tensorflow as tf

from matplotlib import pyplot

from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops

# Required parameters
flags.DEFINE_string("bert_config_file", None, "The config json file corresponding to the pre-trained BERT model. This specifies the model architecture.")

flags.DEFINE_string("task_name", 'PMLP', "The name of the task to train.")

flags.DEFINE_string("vocab_file", None, "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string("output_dir", None, "The output directory where the model checkpoints will be written.")

flags.DEFINE_string("data_dir", None, "The input data dir. Should contain the .tsv files (or other data files) for the task.")

flags.DEFINE_string("init_checkpoint", None, "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool("do_lower_case", True, "Whether to lower case the input text. Should be True for uncased models and False for cased models.")

flags.DEFINE_integer("max_seq_length", 128, "The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")

flags.DEFINE_bool("do_eval", False, "Whether to run eval on the dev set.")

flags.DEFINE_bool("do_predict", True, "Whether to run the model in inference mode on the test set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("eval_batch_size", 8, "Total batch size for eval.")

flags.DEFINE_integer("predict_batch_size", 8, "Total batch size for predict.")

flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0, "Total number of training epochs to perform.")

flags.DEFINE_float("warmup_proportion", 0.1, "Proportion of training to perform linear learning rate warmup for. E.g., 0.1 = 10% of training.")

flags.DEFINE_integer("save_checkpoints_steps", 1000, "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000, "How many steps to make in each estimator call.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

tf.flags.DEFINE_string("tpu_name", None, "The Cloud TPU to use for training. This should be either the name used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 url.")

tf.flags.DEFINE_string("tpu_zone", None, "[Optional] GCE zone where the Cloud TPU is located in. If not specified, we will attempt to automatically detect the GCE project from metadata.")

tf.flags.DEFINE_string("gcp_project", None, "[Optional] Project name for the Cloud TPU-enabled project. If not specified, we will attempt to automatically detect the GCE project from metadata.")

tf.flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_integer("num_tpu_cores", 8, "Only used if `use_tpu` is True. Total number of TPU cores to use.")

#J, my own other parameters
flags.DEFINE_integer("num_classes", 9, "Total number of labels for patent CPC Level 1, A~H & Y")  
flags.DEFINE_string("cpc_id", None, 'CPC labels') 
flags.DEFINE_string("prefix", None, 'CPC prefix') 
flags.DEFINE_integer("times_for_num_train_steps", 1, 'multiply train steps') 
flags.DEFINE_integer("fixed_num_train_steps", -1, 'fixed number of train steps') 
flags.DEFINE_bool("file_based_flag", True, "Whether to read data by file-based")
flags.DEFINE_bool("reuse_tf_record", True, "Whether to reuse TF records")
flags.DEFINE_bool("replace_eval_with_predict_tf_record", False, "Whether to use PREDICT records as DEV records for debugging")
flags.DEFINE_float("multi_hot_threshold", 0.5, "Threshold for making multi-hot-label as 1")
flags.DEFINE_bool("calc_total_by_label", False, "calculate total count by label list")
flags.DEFINE_string("pred_result_file", 'predict_result.txt', 'predict result file') 
flags.DEFINE_string("train_file", 'train.tsv', 'train file') 
flags.DEFINE_string("dev_file", 'dev.tsv', 'dev file') 
flags.DEFINE_string("test_file", 'test.tsv', 'test file') 
flags.DEFINE_string("dev_tf_record", 'eval.tf_record', 'eval tf record') 
flags.DEFINE_integer("label_column", 0, 'which column as label') 
flags.DEFINE_integer("text_column", 3, 'which column as text') 
flags.DEFINE_bool("skip_header", True, "skip header in files")
flags.DEFINE_string("label_to_count", None, 'e.g., A,G,AG') 
flags.DEFINE_integer("number_of_predictions", -1, 'how many predictions to run') 
flags.DEFINE_string("label_file", None, 'label file') 
flags.DEFINE_bool("save_pred_result", True, "save predicton result")

FLAGS = flags.FLAGS

class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example

class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir):
    """Gets a collection of `InputExample`s for prediction."""
    raise NotImplementedError()

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""

    if input_file.startswith('gs://'):
      f = tf.gfile.Open(input_file, "r")
    else:
      f = open(input_file, 'r') #, encoding='utf-8')

    reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
    lines = []
    count = 0
    try:
      for line in reader:
        lines.append(line)
        count += 1
    except:
      print('count: %s' % count)
      pdb.set_trace() 

    print('line count: %s' % count)    
    f.close() 

    return lines

def create_int_feature(values):
  f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
  return f

def get_number_of_txt_file(fn):
  count = 0 

  if fn.startswith('gs://'):
    f = tf.gfile.Open(input_file, "r")
  else:
    f = open(input_file, 'r')

  for i, line in enumerate(f):
    count += 1
      
  if count == 0:
    tf.logging.info('missing file: %s' % fn)
    pdb.set_trace()

  f.close() 

  return count

def pack_one_tf_example(i, example, label_list, max_seq_length, tokenizer):
  feature = convert_single_example(i, example, label_list, max_seq_length, tokenizer)

  feature = convert_single_example(i, example, label_list,
                                   max_seq_length, tokenizer)
  features = collections.OrderedDict()
  features["input_ids"] = create_int_feature(feature.input_ids)
  features["input_mask"] = create_int_feature(feature.input_mask)
  features["segment_ids"] = create_int_feature(feature.segment_ids)

  ## new
  # if feature.label_id is already a list, then no need to add [].
  if isinstance(feature.label_id, list):
    features["label_ids"] = create_int_feature(feature.label_id)
  else:
    # org
    features["label_ids"] = create_int_feature([feature.label_id])
  features["is_real_example"] = create_int_feature(
      [int(feature.is_real_example)])
  tf_example = tf.train.Example(features=tf.train.Features(feature=features))

  return tf_example
  
class PatentMultiLabelProcessor(DataProcessor):
  def get_train_examples(self, data_dir):
    """See base class."""
    tf.logging.info('get_train_examples: %s' % FLAGS.train_file)
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, FLAGS.train_file)), 'train')

  def get_dev_examples(self, data_dir):
    """See base class."""
    tf.logging.info('get_dev_examples: %s' % FLAGS.dev_file)
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, FLAGS.dev_file)), 'dev')

  def get_test_examples(self, data_dir):
    """See base class."""
    tf.logging.info('get_test_examples: %s' % FLAGS.test_file)
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, FLAGS.test_file)), 'test')

  def get_labels(self, cpc_id=None, prefix=None):
    """See base class."""
    lst = []
    #cpc_id = 'section_id'      # A~H, Y
    #cpc_id = 'subsection_id'   # A01
    #cpc_id = 'group_id'        # A01B
    #cpc_id = 'subgroup_id'     # C40B40/14

    if cpc_id == 'section_id':
      lst = [ chr(i) for i in range(ord('A'), ord('H')+1) ] + ['Y']
      return lst

    f = open(FLAGS.label_file)
    tf.logging.info('Reading label file: %s' % FLAGS.label_file)
    reader = csv.reader(f, delimiter="\t")
    lines = []
    for i, line in enumerate(reader):
      if i == 0 and cpc_id != 'ipc':
        continue
      if len(line[0]) == 0:
        continue
      lst.append(line[0])

    f.close() 

    return lst

  def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []

    tf.logging.info('_create_examples(): begins....')
    for (i, line) in enumerate(lines):
      # skip header
      if i == 0 and FLAGS.skip_header == True:
        continue

      if FLAGS.number_of_predictions > 0 and i >= FLAGS.number_of_predictions:
        tf.logging.info('count of predictions (%s)' % i)
        break  

      if i % 5000 == 0:
        tf.logging.info('_create_examples: %s' % i)

      data_ok = False
      label = None
      text_a = None

      tc = FLAGS.label_column
      label = tokenization.convert_to_unicode(line[tc])
      tc = FLAGS.text_column   
      text_a = tokenization.convert_to_unicode(line[tc])

      if len(label) == 0:
        continue        
      guid = "%s-%s" % (set_type, i)
      examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))

    tf.logging.info('_create_examples(): done')
    return examples

def convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""  
  if FLAGS.task_name == 'PMLP':
    if isinstance(example, PaddingInputExample):
      return InputFeatures(
          input_ids=[0] * max_seq_length,
          input_mask=[0] * max_seq_length,
          segment_ids=[0] * max_seq_length,
          label_id=[0] * 9,       #J, bug? 
          #label_id=0,
          is_real_example=False)
  else: # org
    if isinstance(example, PaddingInputExample):
      return InputFeatures(
          input_ids=[0] * max_seq_length,
          input_mask=[0] * max_seq_length,
          segment_ids=[0] * max_seq_length,
          label_id=0,
          is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  ## new
  #print("label_map:",label_map,";length of label_map:",len(label_map))
  label_id=None

  #use task name to check
  #if "," in example.label: # multiple label
  if FLAGS.task_name == 'PMLP':
      # get list of label
      label_id_list=[]
      example_label_list=example.label.split(",")
      for label_ in example_label_list:
          label_id_list.append(label_map[label_])

      # convert to multi-hot style
      label_id=[0 for l in range(len(label_map))]
      for j, label_index in enumerate(label_id_list):
          label_id[label_index]=1
  else: # single label
      # org
      label_id = label_map[example.label]
  ## new

  if ex_index < 1:    # org: 5
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))

    ## new 
    #if ',' in example.label: 
    if FLAGS.task_name == 'PMLP':
      # if label_id is a list, try print multi-hot value: label_id_list
      tf.logging.info("label: %s (id_list = %s)" % (str(example.label), str(label_id_list))) 
    tf.logging.info("label: %s (id = %s)" % (str(example.label), str(label_id))) # %d

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id,
      is_real_example=True)

  return feature

def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  writer = tf.python_io.TFRecordWriter(output_file)
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)

    ## new
    # if feature.label_id is already a list, then no need to add [].
    if isinstance(feature.label_id, list):
      features["label_ids"] = create_int_feature(feature.label_id)
    else:
      features["label_ids"] = create_int_feature([feature.label_id])
    ## new
    ## org
    #features["label_ids"] = create_int_feature([feature.label_id])

    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())

  writer.close()

def file_based_input_fn_builder(input_file, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  if FLAGS.task_name == 'PMLP':
    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([FLAGS.num_classes], tf.int64), 
        "is_real_example": tf.FixedLenFeature([1], tf.int64),
    }
  else:
    # org
    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""

    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32. So cast all int64 to int32.
    if FLAGS.use_tpu:  #J
      for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
          t = tf.to_int32(t)
        example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    #J
    d = d.apply(tf.data.experimental.map_and_batch(
      lambda record: _decode_record(record, name_to_features),
      batch_size=batch_size,
      drop_remainder=drop_remainder))

    # org
    '''
    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))
    '''
    return d

  return input_fn

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

def multi_label_hot(prediction, threshold=0.5):
  prediction = tf.cast(prediction, tf.float32)

  if threshold >= 1:
    # by top n
    top_v = int(FLAGS.multi_hot_threshold) + 1 # 2 as top 1 
    new_scores, new_ids = tf.nn.top_k(prediction, top_v) 
    threshold = tf.reduce_min(new_scores)
  else:
    # by threshold = 0.5
    threshold = float(threshold)

  return tf.cast(tf.greater(prediction, threshold), tf.int64)

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids, labels, num_labels, use_one_hot_embeddings, pos_weight=None):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use model.get_sequence_output()
  # instead.
  output_layer = model.get_pooled_output()
  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))
  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)

    if FLAGS.task_name == 'PMLP':
      # mulit-label classification: 1. multi-hot==> then use sigmoid to transform it to possibility
      probabilities = tf.nn.sigmoid(logits)

      multi_hot_prediction = multi_label_hot(probabilities, FLAGS.multi_hot_threshold)
      tf.logging.info('multi_hot_prediction: %s' % multi_hot_prediction)
      log_probs=tf.log(probabilities)
      labels = tf.cast(labels, tf.float32)
      per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
      loss = tf.reduce_mean(per_example_loss)
      return (loss, per_example_loss, logits, probabilities, multi_hot_prediction, labels)  # labels from input dataset
    else:
      ## org
      probabilities = tf.nn.softmax(logits, axis=-1)
      log_probs = tf.nn.log_softmax(logits, axis=-1)
      one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
      per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
      loss = tf.reduce_mean(per_example_loss)
      return (loss, per_example_loss, logits, probabilities)
   
def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate, num_train_steps, num_warmup_steps, use_tpu,use_one_hot_embeddings, pos_weight):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""
    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None

    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    tf.logging.info('num_labels = %s' % num_labels)

    if FLAGS.task_name == 'PMLP':
      (total_loss, per_example_loss, logits, probabilities, multi_hot_prediction, labels) = create_model(
          bert_config, is_training, input_ids, input_mask, segment_ids, label_ids, num_labels, use_one_hot_embeddings, pos_weight)
    else:
      # org
      (total_loss, per_example_loss, logits, probabilities) = create_model(bert_config, is_training, input_ids, input_mask, segment_ids, label_ids, num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:
        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()
        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info(" name = %s, shape = %s%s", var.name, var.shape, init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:
      if FLAGS.task_name == 'PMLP':
        def metric_fn(per_example_loss, label_ids, logits, is_real_example, multi_hot_prediction):
          accuracy = tf.metrics.accuracy(label_ids, multi_hot_prediction)
          loss = tf.metrics.mean(per_example_loss, weights=is_real_example)
          return {
              "eval_accuracy": accuracy,
              #"eval_accuracy": (accuracy, update_op_accuracy),
              "eval_loss": loss,
          }
        eval_metrics = (metric_fn, [per_example_loss, label_ids, logits, is_real_example, multi_hot_prediction])          
      else:
        # org
        def metric_fn(per_example_loss, label_ids, logits, is_real_example):
          predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
          accuracy = tf.metrics.accuracy(
              labels=label_ids, predictions=predictions, weights=is_real_example)
          loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
          return {
              "eval_accuracy": accuracy,
              "eval_loss": loss,
          }
        eval_metrics = (metric_fn, [per_example_loss, label_ids, logits, is_real_example])

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else: # ModeKeys.PREDICT
      if FLAGS.task_name == 'PMLP':
        output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={'probabilities': probabilities, 'multi_hot_prediction': multi_hot_prediction, 'labels': labels}, scaffold_fn=scaffold_fn)
      else:
        # org
        output_spec = tf.contrib.tpu.TPUEstimatorSpec(
            mode=mode,
            predictions={"probabilities": probabilities},
            scaffold_fn=scaffold_fn)

    return output_spec

  return model_fn


# This function is not used by this file but is still used by the Colab and people who depend on it.
def input_fn_builder_v2(features, seq_length, is_training, drop_remainder, num_label):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []
  all_is_real_example = [] 

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)
    all_is_real_example.append(feature.is_real_example)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]
    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do not use Dataset.from_generator() because that uses tf.py_func which is not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids": 
            tf.constant(all_label_ids, shape=[num_examples, num_label], dtype=tf.int32),
        "is_real_example": 
            tf.constant(all_is_real_example, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]
    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

# This function is not used by this file but is still used by the Colab and people who depend on it.
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,max_seq_length, tokenizer)

    features.append(feature)
  return features

def check_parameters(processors):  
  tokenization.validate_case_matches_checkpoint(FLAGS.do_lower_case, FLAGS.init_checkpoint)

  if not FLAGS.do_train and not FLAGS.do_eval and not FLAGS.do_predict:
    raise ValueError("At least one of `do_train`, `do_eval` or `do_predict' must be True.")

  bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

  if FLAGS.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
      "Cannot use sequence length %d because the BERT model "
      "was only trained up to sequence length %d" % (FLAGS.max_seq_length, bert_config.max_position_embeddings))

  if tf.gfile.Exists(FLAGS.output_dir):
    tf.logging.info('Existed: %s ' % FLAGS.output_dir)
  else:
    tf.gfile.MakeDirs(FLAGS.output_dir)

  task_name = FLAGS.task_name.lower()
  if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))

  return bert_config, task_name

def show_parameters_pmlp(label_list):    
    tf.logging.info('labels: %s' % label_list)
    tf.logging.info('FLAGS.num_classes = %s' % FLAGS.num_classes)
    if len(label_list) == 0:
      tf.logging.info('Empty label list')
      pdb.set_trace()
    tf.logging.info('multi_hot_threshold: %s' % FLAGS.multi_hot_threshold)
     
def configure_tpu():
  tpu_cluster_resolver = None
  if FLAGS.use_tpu and FLAGS.tpu_name:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=FLAGS.iterations_per_loop,
      num_shards=FLAGS.num_tpu_cores,
      per_host_input_for_training=is_per_host))
  tf.logging.info('RunConfig: done')

  return run_config

def save_pred_result(label_list, all_y_lables, 
  all_y_multi_hot_prediction, all_y_probabilities):
  result = []

  reverse_map = {}
  for i, label in enumerate(label_list):
    reverse_map[i] = label

  fn = FLAGS.pred_result_file
  with open(fn, 'w') as f:
    for i, multi_hot in enumerate(all_y_multi_hot_prediction):
      pred_lst = []
      prob_lst = []
      msg = []
      for j, pred_value in enumerate(multi_hot):
        if pred_value == 1:
          pred_lst.append(reverse_map[j])
          prob_lst.append('%.2f' % all_y_probabilities[i][j])
          msg.append('%s (%.2f)' % (reverse_map[j], all_y_probabilities[i][j]))
      f.write('%s\n' % ', '.join(msg) )    

  tf.logging.info('saved: %s' % fn)

def show_predict_result_pmlp(label_list, result, num_actual_predict_examples):
  # label_list: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']
  all_y_lables = []
  all_y_multi_hot_prediction = []
  all_y_probabilities = []
  tf.logging.set_verbosity(tf.logging.WARN)  # avoid INFO message 
  tf.logging.info("***** Predict results *****")
  try:
    for (i, prediction) in enumerate(result):
      if i % 1000 == 0:
        tf.logging.warning('i = %s / %s' % (i, num_actual_predict_examples))

      if i >= num_actual_predict_examples:
        break

      all_y_lables.append(prediction['labels'])
      all_y_multi_hot_prediction.append(prediction['multi_hot_prediction'])
      all_y_probabilities.append(prediction['probabilities'])
  except Exception as e:
    tf.logging.warning('%s' % str(e))
    tf.logging.warning('(my own log) Assuming this is caused by padded examples saved in tf records and should be avoided in non-TPU mode')
  tf.logging.set_verbosity(tf.logging.INFO)

  if FLAGS.save_pred_result == True:
    save_pred_result(label_list, all_y_lables, 
      all_y_multi_hot_prediction, all_y_probabilities)

def run_predict(estimator, predict_examples, label_list, tokenizer, task_name):
  tf.logging.info("***** Running prediction*****")
  tf.logging.info("  Batch size = %d", FLAGS.predict_batch_size)
  tf.logging.info("  file_based_flag = %s" % FLAGS.file_based_flag)
  num_actual_predict_examples = len(predict_examples)

  if FLAGS.use_tpu:
    while len(predict_examples) % FLAGS.predict_batch_size != 0:
      predict_examples.append(PaddingInputExample())

  if FLAGS.file_based_flag == True:
    predict_file = os.path.join(FLAGS.output_dir, "predict.tf_record")
    if FLAGS.reuse_tf_record == True and tf.gfile.Exists(predict_file):
      tf.logging.info('Reuse: %s' % predict_file)
    else:
      file_based_convert_examples_to_features(predict_examples, label_list, FLAGS.max_seq_length, tokenizer, predict_file)
    tf.logging.info("  Num examples = %d (%d actual, %d padding)",len(predict_examples), num_actual_predict_examples,len(predict_examples) - num_actual_predict_examples)
    predict_drop_remainder = True if FLAGS.use_tpu else False
    predict_input_fn = file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=FLAGS.max_seq_length,
        is_training=False,
        drop_remainder=predict_drop_remainder)
  else:
    features = convert_examples_to_features(predict_examples, label_list, FLAGS.max_seq_length, tokenizer)
    predict_input_fn = input_fn_builder_v2(
      features=features,
      seq_length=FLAGS.max_seq_length,
      is_training=False,
      drop_remainder=True,
      num_label=len(label_list))

  result = estimator.predict(input_fn=predict_input_fn)

  if task_name == 'pmlp':
    show_predict_result_pmlp(label_list, result, num_actual_predict_examples)
  else:
    tf.logging.info('not implemented yet')
    pdb.set_trace()

  tf.logging.info('predict: done')

def main(_):
  tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)    

  processors = {
    "pmlp":PatentMultiLabelProcessor, 
  }

  bert_config, task_name = check_parameters(processors)
  processor = processors[task_name]()

  if task_name == 'pmlp':
    label_list = processor.get_labels(FLAGS.cpc_id, FLAGS.prefix)
    FLAGS.num_classes = len(label_list)
    show_parameters_pmlp(label_list)
  else:
    # org
    label_list = processor.get_labels()

  tokenizer = tokenization.FullTokenizer(vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)
  run_config = configure_tpu()

  train_examples = num_train_examples = num_train_steps = pos_weight = num_warmup_steps = None

  if FLAGS.do_predict:
    predict_examples = processor.get_test_examples(FLAGS.data_dir)

  model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=FLAGS.init_checkpoint,
      learning_rate=FLAGS.learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=FLAGS.use_tpu,
      use_one_hot_embeddings=FLAGS.use_tpu,
      pos_weight=pos_weight)

  # If TPU is not available, this will fall back to normal Estimator on CPU or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size,
    )

  if FLAGS.do_predict:
    run_predict(estimator, predict_examples, label_list, tokenizer, task_name)

if __name__ == "__main__":
  flags.mark_flag_as_required("task_name")
  flags.mark_flag_as_required("vocab_file")
  flags.mark_flag_as_required("bert_config_file")
  flags.mark_flag_as_required("output_dir")
  tf.app.run()
  

Writing patent_classifier.py


In [ ]:
%%writefile test_data.tsv
group_ids	id	date	text
C10J,C10L,Y02E	9062268	2015-06-23	A method of producing an engineered fuel feed stock from a processed waste stream, the method comprising the steps of:a) selecting a plurality of components from a processed waste stream which components in combination have chemical molecular characteristics comprising:a carbon content of between about 30 wt. % and about 80 wt. %, anda hydrogen content of between about 3 wt. % and about 10 wt. %; andb) combining the selected components of step a) to form an engineered fuel feed stock; wherein the engineered fuel feed stock contains biodegradable waste and non-biodegradable waste and is substantially free of glass, metals, grit, and noncombustible waste.
G01K,H04Q	8981943	2015-03-17	A wireless temperature measuring system,comprising:multiple wireless temperature sensors; andmultiple communication terminals;whereinsaid wireless temperature sensors communicate with said communication terminals via radio frequency (RF);said wireless temperature sensors operate to obtain temperature signals from a temperature detection point, to perform cross-interleaved error correction encoding on said temperature signals whereby obtaining coded temperature signals, and to transmit said coded temperature signals to said communication terminals via RF;said communication terminals operate to receive and decode said coded temperature signals from said wireless temperature sensors, and to obtain decoded temperature signals;each of said wireless temperature sensors comprises a temperature collection module, an analog/digital (A/D) conversion module, a first micro-controller unit (MCU) module, and a first RF module;said temperature collection module operates to collect temperature of said temperature detection point whereby obtaining an analog temperature signal, to transmit said analog temperature signal to said A/D conversion module, and to transmit an ID number of said temperature collection module to said first MCU module;said A/D conversion module operates to receive said analog temperature signal from said temperature collection module, to perform A/D conversion on said analog temperature signal whereby obtaining a digital temperature signal, and to transmit said digital temperature signal to said first MCU module;said first MCU module operates to receive said digital temperature signal from said A/D conversion module, and said ID number of said temperature collection module from said temperature collection module, to pack said digital temperature signal and said ID number of said temperature collection module with data frames whereby obtaining a temperature signal data packet, to perform cross-interleaved error correction encoding on said temperature signal data packet whereby obtaining a coded temperature signal, and to transmit said coded temperature signal to said first RF module;said first RF module operates to receive said coded temperature signal from said first MCU module, and to transmit said coded temperature signal to said communication terminal;each of said multiple communication terminals comprises a second RF module, a second MCU module, and a display module;said second RF module operates to receive said coded temperature signal from said first RF module, and to transmit said coded temperature signal to said second MCU module;said second MCU module operates to receive said coded temperature signal from said second RF module, to decode said coded temperature signal whereby obtaining a decoded temperature signal, and to transmit said decoded temperature signal to said display module;said display module operates to receive and to display said decoded temperature signal from said second MCU module;said first MCU module further operates to detect whether an RF signal exists at a main frequency point of said first RF module after receiving said coded temperature signal, to transmit said coded temperature signal to said first RF module and to inform said first RF module to transmit said coded temperature signal via said main frequency point if no RF signal exists at a main frequency point of said first RF module, or to transmit said coded temperature signal to said first RF module and to inform said first RF module to transmit said coded temperature signal via an auxiliary frequency point if said RF signal exists at a main frequency point of said first RF module;said second MCU module further operates to set values of said main frequency point and said auxiliary frequency point, to pack said values of said main frequency point and said auxiliary frequency point with data frames whereby obtaining a frequency point packet, to perform cross-interleaved error correction encoding on said frequency point packet whereby obtaining a coded frequency point, and to transmit said coded frequency point to said second RF module;said second RF module further operates to receive said coded frequency point from said second MCU module, and to transmit said coded frequency point to said first RF module via RF;said first RF module further operates to receive said coded frequency point from said second RF module, and to transmit said coded frequency point to said first MCU module; andsaid first MCU module further operates to receive said coded frequency point from said first RF module, to decode said coded frequency point whereby obtaining values of said main frequency point and said auxiliary frequency point, and to set said main frequency point and said auxiliary frequency point of said first RF module.
H04W	9137724	2015-09-15	A method, comprising:receiving at least one signal indicative that (1) a first access point is configured to transmit data over a maximum number of spatial streams equal to a first number of spatial streams, and (2) a second access point is configured to transmit data over a maximum number of spatial streams equal to a second number of spatial streams less than the first number of spatial streams;receiving a probe request from each client device from a plurality of client devices, a first portion of the plurality of client devices configured to transmit data over a maximum number of spatial streams equal to the first number of spatial streams, a second portion of the plurality of client devices configured to transmit data over a maximum number of spatial streams equal to the second number of spatial streams;sending a signal to each client device from the first portion of the plurality of client devices such that that client device is authorized to connect to a network via the first access point; andsending a signal to each client device from the second portion of the plurality of client devices such that that client device is authorized to connect to the network via the second access point based on the second portion of the plurality of client devices being configured to transmit data over the maximum number of spatial streams equal to the second number of spatial streams.
C07C,C08G,C08L,F21V,G02B,H05B	9221949	2015-12-29	A resin comprising polymer chains having an acylhydrazone bond and aldehyde groups or acylhydrazine groups at their ends, whereinthe resin has a number average molecular weight of 500 to 500,000,the resin has an acylhydrazone bond equivalent of 100 to 4,000,at least some polymer chains of the resin are end-capped at their aldehyde end or acylhydrazine end, andthe polymer chains comprise a repeating unit represented by the following Formula (1):wherein in Formula (1),X1 represents a divalent linking group containing atoms selected from the group consisting of C, H, N, O, S, Si, F, Cl, Br and I and having a molecular weight of 80 to 8,000, in which the number of atoms of the shortest molecular chain that links the carbonyl carbon atoms of two acylhydrazone bonds is 2 to 60;R3 and R4 each independently represent a hydrogen group, a methyl group, or a phenyl group;Z represents a group represented by any one of the following Formulas (2) to (6), or a saturated hydrocarbon group having 1 to 12 carbon atoms:wherein in Formula (2),A1 represents &#x2014;CH&#x2550; or &#x2014;N&#x2550;, while the positions of the bonding hands on the aromatic ring are the 2-position and the 6-position, or the 2-position and the 5-position with respect to A1, and the aromatic ring may have a methyl group, an ethyl group or a halogen atom at the other positions;in Formula (3), A2 represents &#x2014;CH2&#x2014;, &#x2014;O&#x2014;, &#x2014;S&#x2014;, or &#x2014;N(R)&#x2014; (wherein R represents a hydrogen atom, a methyl group, or an ethyl group);in Formula (4), R5 represents a hydrogen atom, a methyl group, an ethyl group, a methoxy group, an ethoxy group, a halogen atom, a methylthio group, or an ethylthio group;in Formula (5), A3 represents &#x2014;CH2&#x2014;, &#x2014;C(CF3)2&#x2014;, &#x2014;S&#x2014;, &#x2014;C(CH3)2&#x2014;, &#x2014;O&#x2014;, &#x2014;SO2&#x2014;, &#x2014;S(&#x2550;O)&#x2014;, &#x2014;C(&#x2550;O)&#x2014;, &#x2014;C(&#x2550;O)NH&#x2014;, or &#x2014;O&#x2014;CpH2p&#x2014;O&#x2014; (wherein p represents an integer from 2 to 18); andin Formula (6), 1 represents an integer from 1 to 8.
F41A,F41C	8991089	2015-03-31	A ramrod for a muzzleloader, the ramrod comprising:a shaft having a distal end; anda knob having a first end, a second end that opposes the first end, and a side portion defined between the first and second ends, the first end of the knob being releasably coupled to the distal end of the shaft, the first end, the second end, and the side portion being fixedly positioned with respect to the shaft once the knob is coupled to the shaft, the knob defining an indentation surface in the side portion, and wherein the indentation surface is arc-shaped and is oriented axially along the length of the knob, and the arc-shaped indentation surface comprises a longitudinal axis that is substantially parallel with an axis of the shaft when stored and when in use;wherein the shaft is configured to releasably couple to the muzzleloader and,with the shaft coupled to the muzzleloader,the indentation surface of the knob functionally engages a barrel of the muzzleloader with a majority of the indentation surface physically contacting the barrel; andwherein the indentation surface is an interchangeable insert that is configured to releasably couple to the knob.
E21B	9151131	2015-10-06	A method of operating an electric submersible pump (ESP) in a subsea wellbore, comprising:supplying a direct current (DC) power signal from a dry location to a subsea control pod;converting the DC power signal to a three phase alternating current (AC) power signal by the control pod; andsupplying the three phase AC power signal from the control pod, through a subsea production tree, into the subsea wellbore, and to the ESP,wherein:the ESP pumps production fluid from a reservoir intersected by the wellbore to the subsea tree via production tubing, andthe three phase AC power signal is routed laterally through the subsea tree such that crown plugs of the subsea tree remain in place during pumping of the production fluid.
G06F	9223927	2015-12-29	A modeling system, comprising:a processor including a capacitor model generator configured to generate a capacitor model based on a received circuit configuration,the capacitor model generator including,an extract module configured to extract parasitic capacitors from the received circuit configuration; anda generate module configured to generate the capacitor model by,classifying the parasitic capacitors into a group of coupled capacitors and a group of grounded capacitors,classifying the coupled capacitors into first coupled capacitors and second coupled capacitors according to a corresponding influence on a performance of the circuit,setting the first coupled capacitors to a maintenance state, andconverting at least one of the second coupled capacitors into a grounded capacitor, the at least one of the second coupled capacitors being a second coupled capacitor having a capacitance that is below a desired reference value.
A61B,A61N	9138160	2015-09-22	An apparatus for detection and treatment of atrial arrhythmia, the apparatus comprising:an electrical lead having a proximal and distal portion, the distal portion being positionable in an atrial chamber of a heart, the proximal portion having an end configured to be coupled to a pacemaker device, the lead comprising a plurality of conductive wires;a plurality of membrane patches distributed along the electrical lead, each membrane patch configured to be attached to a wall of the heart and to bend and flex with motion of the heart without impeding motion of the heart; anda plurality of pairs of electrodes, wherein at least one pair of electrodes is positioned on each membrane patch, the electrodes of each pair being (i) coupled to a conductive wire of the lead, and (ii) configured to at least sense an electrical signal of the wall of the heart; andwherein the distribution of the plurality of membrane patches and electrode pairs along the lead is configured to allow the plurality of electrode pairs to be distributed in a pattern on the heart wall, the pattern defining an area for i) detecting a location of a foci of aberrant electrical activity located within or adjacent to the area, and ii) sending a pacing signal to that location to prevent or stop an occurrence of atrial fibrillation caused by that foci.
G06T,H04N	8934044	2015-01-13	A method comprising:in response to receiving, by an image editing application, selection of a layer from an image comprising at least two layers, selecting the layer in the image editing application;rendering, by the image editing application, a view from a camera in at least a portion of the selected layer;rendering, by the image editing application, a camera fill view, the camera fill view simultaneously including at least the layer from the image, the view from the camera, and a menu allowing a user to control an operation of the camera;in response to receiving, by the image editing application, an input indicating that the view from the camera should be captured, capturing the view from the camera;inserting, by the image editing application, the captured view from the camera into the at least the portion of the selected layer; andrendering, by the image editing application, a message corresponding to the inserted captured view and indicating the layer in which the captured view is located.
G06F	9223996	2015-12-29	A method comprising:storing information identifying, for each of a plurality of programs, a correspondence between a first area of memory in which is stored executable instructions of the program, a second area of memory in which is stored protected data to be processed by the program, and a third area of memory in which is stored shared data to be processed by the program;with a second circuit different from an execution unit to execute instructions of the plurality of programs, monitoring instructions executed by the execution unit and, when an instruction is to access data stored in memory, jointly analyzing a first address at which the instruction is stored and the information identifying the correspondence to determine whether the first address is in an area of memory corresponding to an area in which the data to be accessed is stored, the second circuit configured to produce a hit status when access to a particular area of memory is valid and the second circuit configured to produce a no-hit status when access to the particular area of memory is a violation;based on the monitoring, producing at least one second-area signal representing a second-area hit status or a second-area no-hit status associated with the second area of memory, and producing at least one third-area signal representing a third area hit status or a third area no-hit status associated with the third area of memory; andpreventing access to the data stored in memory when at the least one second-area signal represents a second-area no-hit status or when the at least one third-area signal represents a third-area no-hit status.
H01L	8963298	2015-02-24	An electromagnetic interference (EMI) shielding package structure, comprising:a package substrate having a first surface with a die mounting area and a second surface opposite to the first surface;a plurality of metallic pillars disposed on the first surface;a chip mounted on and electrically connected to the die mounting area;an encapsulant for encapsulating the chip and the first surface of the package substrate, in a manner that a portion of each of the metallic pillars is exposed from the encapsulant; anda shielding film enclosing the encapsulant, the metallic pillars, and a part of sides of the package substrate and electrically connected to the metallic pillars, wherein an outer side of the shielding film is flush with the other part of the sides of the package substrate, and wherein the other part of the sides of the package substrate is free from being enclosed by the shielding film.
A61K,C07C,C07F,C07K	9175018	2015-11-03	A process for generating a compound of Formula (II)or a pharmaceutically acceptable salt thereof, whereinA is 0, 1, or 2;P is Rc&#x2014;C(O)&#x2014;; Rc is RD; RD is substituted or unsubstituted mono- or bicyclic ring system selected from the group consisting of phenyl, pyridinyl, pyrimidinyl, naphthyl, benzimidazolyl, quinolinyl, isoquinolinyl, quinoxalinyl, tetrahydroquinolinyl, tetrahydroisoquinolinyl, tetrahydroquinoxalinyl, and dihydrobenzoxazinyl;Ra is hydrogen, C1-6 aliphatic,C1-6 fluoroaliphatic, &#x2014;(CH2)m&#x2014;CH2&#x2014;RB, &#x2014;(CH2)m&#x2014;CH2&#x2014;NHC(&#x2550;NR4)NH&#x2014;Y, &#x2014;(CH2)m&#x2014;CH2&#x2014;CON(R4)2, &#x2014;(CH2)m&#x2014;CH2&#x2014;N(R4)CON(R4)2, &#x2014;(CH2)m&#x2014;CH(R6)N(R4)2, &#x2014;(CH2)m&#x2014;CH(R5a)&#x2014;OR5b,or &#x2014;(CH2)m&#x2014;CH(R5)&#x2014;SR5;Ra1 is hydrogen, C1-6 aliphatic,C1-6 fluoroaliphatic, &#x2014;(CH2)m&#x2014;CH2&#x2014;RB, &#x2014;(CH2)m&#x2014;CH2&#x2014;NHC(&#x2550;NR4)NH&#x2014;Y, &#x2014;(CH2)m&#x2014;CH2&#x2014;CON(R4)2, &#x2014;(CH2)m&#x2014;CH2&#x2014;N(R4)CON(R4)2, &#x2014;(CH2)m&#x2014;CH(R6)N(R4)2, &#x2014;(CH2)m&#x2014;CH(R5a)&#x2014;OR5b,or &#x2014;(CH2)m&#x2014;CH(R5)&#x2014;SR5;each Ra2 independently is hydrogen, C1-6 aliphatic,C1-6 fluoroaliphatic, &#x2014;(CH2)m&#x2014;CH2&#x2014;RB, &#x2014;(CH2)m&#x2014;CH2&#x2014;NHC(&#x2550;NR4)NH&#x2014;Y, &#x2014;(CH2)m&#x2014;CH2&#x2014;CON(R4)2, &#x2014;(CH2)m&#x2014;CH2&#x2014;N(R4)CON(R4)2, &#x2014;(CH2)m&#x2014;CH(R6)N(R4)2, &#x2014;(CH2)m&#x2014;CH(R5a)&#x2014;OR5b,or &#x2014;(CH2)m&#x2014;CH(R5)&#x2014;SR5;each RB independently is a substituted or unsubstituted mono- or bicyclic ring system;each R4 independently is hydrogen or a substituted or unsubstituted aliphatic, aryl, heteroaryl, or heterocyclyl group; or two R4 on the same nitrogen atom, taken together with the nitrogen atom, form a substituted or unsubstituted 4- to 8-membered heterocyclyl ring having, in addition to the nitrogen atom, 0-2 ring heteroatoms independently selected from N, O, and S;each R5 independently is hydrogen or a substituted or unsubstituted aliphatic, aryl, heteroaryl, or heterocyclyl group;each R5a independently is hydrogen or a substituted or unsubstituted aliphatic, aryl, heteroaryl, or heterocyclyl group;each R5b independently is hydrogen or a substituted or unsubstituted aliphatic, aryl, heteroaryl, or heterocyclyl group;each R6 independently is a substituted or unsubstituted aliphatic, aryl, or heteroaryl group;Y is hydrogen, &#x2014;CN, or &#x2014;NO2;m is 0, 1, or 2;each of Rb1 and Rb2 independently is hydrogen, &#x2014;CO2H, &#x2014;OH, or a substituted or unsubstituted aliphatic, aryl, heteroaryl, or heterocyclyl group;each of Rb3 and Rb4 independently is hydrogen, &#x2014;CO2H, or a substituted or unsubstituted aliphatic, aryl, heteroaryl, or heterocyclyl group;or Rb2 and Rb4 are each independently hydrogen, and Rb1 and Rb3, taken together with the carbon atoms to which they are attached, form an unsubstituted or substituted fused 4- to 8-membered non-aromatic ring having 0-3 ring heteroatoms selected from the group consisting of O, N, and S, wherein said ring may be optionally fused to an unsubstituted or substituted 4- to 8-membered non-aromatic ring, or 5- to 6-membered aromatic ring having 0-3 ring heteroatoms selected from the group consisting of O, N, and S;or Rb2 and Rb4 are absent, and Rb1 and Rb3, taken together with the carbon atoms to which they are attached, form an unsubstituted or substituted fused 5- to 6-membered aromatic ring having 0-3 ring heteroatoms selected from the group consisting of O, N, and S, wherein said ring may be optionally fused to an unsubstituted or substituted 4- to 8-membered non-aromatic ring, or 5- to 6-membered aromatic ring having 0-3 ring heteroatoms selected from the group consisting of O, N, and S; andn is 0 or 1;the process comprising:(1a) coupling a compound of Formula (vi) with a compound, P-LG, to form a compound of Formula (vii)wherein PG is hydrogen or a protecting group; and LG is a leaving group;(2a) deprotecting the compound of Formula (vii) to form a compound of Formula (viia)(3a) coupling the compound of Formula (viia) with a compound of Formula (i) to form a compound of Formula (iv)wherein X&#x2212; is a counter anion;(4) deprotecting the compound of Formula (iv) to form a compound of Formula (v)and(5) reacting the compound of Formula (v) with an alpha hydroxy carboxylic acid or a beta hydroxy carboxylic acid to form a compound of Formula (II).
B01D,B82Y,C23C,Y02C	9056282	2015-06-16	A membrane, comprising:a graphene layer perforated by a plurality of nanoscale pores; anda gas sorbent comprising a material that contacts a surface of the graphene layer such that at least a portion of the nanoscale pores are occluded by the material, wherein the material adsorbs at least one gas and is permeable to the adsorbed at least one gas such that the gas sorbent is configured to direct the at least one gas into the nanoscale pores including the occluded nanoscale pores, and wherein a hydrogen and methane separation selectivity of the membrane is between about 200:1 and about 10^23:1.
B82Y,C04B,C08G,C08J,C08K	9133319	2015-09-15	A method comprising:adding a pre-exfoliated inorganic clay to an isocyanate, said pre-exfoliated inorganic clay is free from modification by an organic counter ion and has not been introduced to an aqueous acidic medium;causing the inorganic clay to delaminate to form a clay nanodispersion;mixing said clay nanodispersion with an isocyante-reactive component and a blowing agent to form a reactive mixture; andallowing the reactive mixture to foam thereby forming a rigid polyurethane foam having a closed cell content greater than 80%.
H01M	9112234	2015-08-18	A cylindrical battery comprising:(a) an electrode assembly and an electrolyte in a cylindrical can, the electrode assembly having a cathode, anode, and separator rolled together;(b) a cap assembly mounted on the opening top of the cylindrical can, said cap assembly comprises:a safety vent, which breaks under increased pressure,wherein the safety vent has a recess located in a center of the safety vent, said recess having an upper bend and a lower bend,said upper bend has a first notch,wherein said first notch has a first arcuate shape region and a first non-notched region, andsaid lower bend has a second notch,wherein the second notch has a second arcuate shaped region, a third arcuate shaped region, a second non-notched region, and a third non-notched region, wherein said second non-notched region and said third non-notched region are located diametrically opposite to one another;a current interruptive device, which upon activation interrupts current,wherein the current interruptive device includes:a protrusion protruding upwardly, said protrusion welded to the bottom of a recess of the safety vent, said protrusion provided at a center of the current interruptive device,a cathode lead connected to the cathode through the bottom of the current interruptive device in a region other than the protrusion; anda bridge, provided with first through holes and a third notch, said third notch connecting the adjacent first through holes,wherein said first through holes and said third notch concentrically surround the center of the protrusion,two or more second throughholes to allow exhaustion of gas,wherein the second through holes have a total area of 20 to 50% with respect to the total area of the current interruptive device,wherein the second through holes are arranged such that the second through holes are spaced from one another by a distance in the form of a circular arc with a length corresponding to an angle of 50 to 80 degrees, based on the center of the current interruptive device; and(c) a gasket surrounding the periphery of thecurrent interruptive device.
G01C,H01L	9048153	2015-06-02	An integrated image sensor capable of determining the distance to at least one object contained in a scene, the sensor comprisingat least a set of first pixels and a set of second pixels, the first and second pixels being alternately distributed in an array, the first pixels having a different angular aperture than the second pixels,the sensor being configured to compare a first sub-image obtained by the set of first pixels with a second sub-image obtained by the set of second pixels, andthe sensor being configured to determine a size difference between at least one object of the scene contained in the first and second sub-images and to deduce therefrom the distance to the at least one object,wherein the angular aperture of each first pixel denotes a maximum angle between two beams of non-zero incidence passing through an optical axis of a first microlens associated with said first pixel to reach a photodetection area of said first pixel,wherein the angular aperture of each second pixel denotes a maximum angle between two beams of non-zero incidence passing through an optical axis of a second microlens associated with said second pixel to reach a photodetection area of said second pixel.
A61H	9138373	2015-09-22	A massager capable of simultaneously applying to an area of a user's body vibrations of different strengths and movements, comprising:a body that extends longitudinally from a first end to a second end;an electrically-powered vibrating element in vibratory communication with the second end;the second end comprising two opposing and spaced-apart flexibly-compliant wings non-rigidly attached with and extending longitudinally away from the body, each of the wings comprising a base connected to the body and a distal end, the base having a cross-section with a first dimension and a second dimension orthogonal to the first dimension, and the first and second dimension tapering to the distal end, wherein the distal end is adapted to flutter by amplifying the movement of the electrically-powered vibrating element; andthe second end further comprising a central end portion located between the wings and adapted to vibrate with the movement of the electrically-powered vibrating element without amplifying the movement of the electrically-powered vibrating element.
B60Q,B64D,Y02T	9174734	2015-11-03	An aircraft storage bin comprising:an upper housing that includes first and second side panels, wherein the first and second side panels each include a bottom edge,a bucket pivotally connected to the upper housing that cooperates with the upper housing to define a bin interior, wherein the bucket includes a bottom and first and second opposing side walls extending upwardly from the bottom, and wherein the first and second side walls of the bucket each include a top edge,a first pivot mechanism operatively associated with the first side wall of the bucket, anda second pivot mechanism operatively associated with the second side wall of the bucket, wherein the bucket pivots about a pivot axis with respect to the upper housing between an open position and a closed position, and wherein, in the closed position, at least a portion of the top edge of the first side wall of the bucket abuts the bottom edge of the first side panel, and at least a portion of the top edge of the second side wall of the bucket abuts the bottom edge of the second side panel.
G06F	8990613	2015-03-24	A method comprising:providing a base image of a primary system to a remote backup service;providing a first plurality of delta images of the primary system to the remote backup service, each delta image of the first plurality of delta images representing an incremental change in a state of the primary system;updating the base image at the remote backup service to create an updated image that reflects the incremental change represented by each of the first plurality of delta images;deactivating the primary system;activating the updated image at the remote backup service; andreceiving, from the remote backup ser rice, a second plurality of delta images at a local backup system, the local backup system being local to the primary system, each of the second plurality of delta images representing an incremental change in a state of the updated image, at the remote backup service.
F16B,Y10T	9175708	2015-11-03	A hole-forming and thread-forming screw with a screw head that has a screw-driving formation for rotational driving by a screwdriver or tool at high speed, a screw shaft that has a threaded groove section and a hole-forming section extending from the threaded groove section and having a thread-free portion extending towards a thread free screw tip, wherein a transverse cross-section of the hole-forming section continuously decreases as it extends from a full diameter of the screw shaft in a direction towards the screw tip, further wherein the hole-forming section includes a first sideline section having a first curvature, a second sideline section having a second curvature different from the first curvature, and an indenture formed by the transition from the first curvature to the second curvature, wherein the first and second sideline sections are directly adjacent to each other;wherein the screw shaft has a threaded edge inscribed along the threaded groove section of the screw shaft, the threaded groove section extending to the hole-forming section of the screw shaft, the threaded groove section and the threaded edge having a transverse cross-section in a form of a polygon, particularly a triangle with rounded corners and convex curved sides; andwherein the screw shaft has a holding section extending towards the screw head, wherein the threaded groove section extends from the holding section to the hole-forming section, and wherein the transverse cross-section of the screw shaft in the holding section is formed to be closer to circular than the polygonal cross-section of the threaded groove section.
G06F	9122734	2015-09-01	A method, comprising:monitoring, on a computing device, data events corresponding to manipulation of data by an application, the monitoring performed by a data library service that is embedded in the application; andin response to the monitoring indicating first data on the computing device is modified by the application, synchronizing, by the computing device under control at least in part of the data library service, second data stored on one or more storage providers in a network with the first data modified by the application and stored on the computing device.
A61B	9149246	2015-10-06	A method for adaptively correcting exposure parameters during digital radiographic imaging, comprising:acquiring a photographic position and a body type set by a user;loading a set of default exposure parameters including a default exposure dose according to the photographic position and the body type, the default exposure parameters being based on a standard optimum Source Image Distance (SID); andacquiring a current SID and adjusting the default exposure dose according to the current SID to obtain an adjusted exposure dose;wherein the adjusted exposure dose is adjusted to be greater than the default exposure dose when the current SID is greater than the standard optimum SID and less than the default exposure dose when the current SID is less than the standard optimum SID.
G01C,G01S	9074897	2015-07-07	A mobile device comprising:a receiver to receive RF signals;a memory; anda special purpose computing device adapted to operate in an RF environment to:process information, represented by first signals, obtained in a first time period to obtain one or more measurements associated with said first time period;store unprocessed information obtained in said first time period;compute an estimate and/or a prediction of a state following said first time period based, at least in part, on said one or more measurements, wherein said state comprises a position and/or trajectory;obtain additional information, represented by second signals, subsequent to said first time period;re-process at least a portion of said stored unprocessed information obtained in said first time period combined with said additional information obtained subsequent to said first time period to obtain updates to said one or more measurements associated with said first time period; andre-compute said estimate and/or prediction of said state that was estimated and/or predicted based on the one or more measurements obtained in the first time period based, at least in part, on said updates obtained by the re-processing of the stored unprocessed information obtained in said first time period combined with said additional information obtained subsequent to said first time period.
E04H,F41H,F42D	9016668	2015-04-28	A barrier comprising:a housing having an exterior surface and an interior surface, the exterior surface comprising a front face and an opposing back face each extending between a top face and an opposing bottom face and also between a first side face and an opposing second side face, the interior surface bounding a chamber that is adapted to receive a ballast, the chamber being bounded by the front face, the opposing back face, the top face, the opposing bottom face, the first side face and the opposing second side face; andan elongated first locking groove recessed on the front face and extending between the top face and the opposing bottom face, the first locking groove including an elongated channel extending between the top face and the bottom face and a constricted mouth through which the channel can be accessed;an elongated further locking groove recessed on the first side face and extending between the top face and the opposing bottom face, the further locking groove including an elongated channel extending between the top face and the bottom face and a constricted mouth through which the channel can be accessed; anda first projection outwardly extending from the top face of the housing, the first projection bounding a port extending therethrough that communicates with the chamber.
G01S,H01P,H01Q	9134400	2015-09-15	A comparator of a mono-pulse radar, comprising:an antenna array, comprising N antennas, wherein N is an integer greater than 1;a transverse electromagnetic (TEM) mode cavity power combiner, comprising a combination port and M input port sets, wherein each of the input port sets has a positive input port and a negative input port, wherein M is an integer greater than 1; anda switch device, coupled between the antenna array and the TEM mode cavity power combiner, and transmitting reflected signals received by each of the antennas to the positive input port or the negative input port of the input port sets,wherein the combination port of the TEM mode cavity power combiner generates an output signal according to the reflected signals received from the input port sets.
H04L,H04W	9094270	2015-07-28	A method comprising:modulating, by at least one modulator at a device, broadcast content to produce modulated broadcast content;modulating, by a plurality of modulators at the device, narrowcast content to produce modulated narrowcast content, wherein the plurality of modulators are separate from the at least one modulator;routing the modulated broadcast content and the modulated narrowcast content to at least one interface included in the device; andtransmitting the modulated broadcast content and the modulated narrowcast content from the at least one interface.
G06Q,G07B,G07C,G07F,H04N	9118802	2015-08-25	An apparatus implementing a Video Ticket Office (VTO) for providing an interactive agent-assisted transportation service, the apparatus comprising:a video display screen oriented to display in the direction of an expected user location;a video camera oriented to obtain images of the expected user location;a microphone oriented to receive sound from the expected user location;a speaker system oriented to direct sound to the expected user location;at least one input device;at least one output device;a communication interface module capable of transmitting and receiving real-time video and audio communications with a separately located operator center;an auxiliary visual display screen comprising a touch screen and configured to display prompts received from the operator center and to receive inputs and cause the inputs to be transmitted to the operator center; anda computer comprising memory and one or more processors communicatively linked to the video display screen, the video camera, the microphone, the speaker system, the at least one input device, the at least one output device and the communication interface module;wherein the processors execute instructions from the memory that cause at least one output device to transmit to the expected user a transportation service output, wherein the instructions further cause the processors to display on the auxiliary visual display screen information regarding service options, the information regarding service options being selected by a service agent at the operator center based on interaction with the user, and to transmit to the operator center a response, entered through the touch screen of the auxiliary display screen, regarding the service options displayed on the auxiliary display screen.
G10D	9159303	2015-10-13	A stringed musical instrument comprising an apparatus for adjusting string height of the stringed instrument, the stringed instrument having a body, a headstock, and a neck interconnecting the headstock and the body, and a nut on the neck for providing support for the strings, comprising:a) an assembly, said assembly capable of being mounted in the neck, said assembly being configured to raise and lower strings of the stringed instrument;b) said assembly comprising a housing, said housing having a channel therein;c) a receptacle disposed on an upper surface of said housing, said receptacle for receiving a nut;d) a sliding member disposed within said housing having an upper sloped surface on which a bottom of said nut can rest, wherein said sliding member is movable within said channel;e) an adjustment member attached to a bottom of said sliding member; and,f) said adjustment member being movable rearwardly and forwardly along with said sliding member to raise and lower said nut riding on said sloped surface to raise and lower the strings for producing different string actions of the stringed instrument.
H01L	9190351	2015-11-17	A packaged integrated circuit (IC) device, comprising:a lead frame having a plurality of leads, including first and second leads;a case encapsulating a first portion of the lead frame, wherein:each of the first and second leads has (i) an encapsulated portion covered by the case and (ii) an exposed portion that protrudes from the case;the exposed portion of each of the first and second leads has an exposed flat segment proximal to the case;the first lead is adjacent to the second lead; andthe exposed flat segment of the first lead is parallel to the exposed flat segment of the second lead; anda first lead webbing located between the exposed flat segment of the first lead and the exposed flat segment of the second lead,wherein a thickness of the first lead webbing is within 20% of a thickness of the first lead.
G06F,H01L	8943455	2015-01-27	A method, comprising:defining standard cells including at least one transistor and polysilicon dummy structures formed on cell edges;forming a pre-layout schematic netlist from an input gate level netlist using the standard cells, wherein the pre-layout schematic netlist includes a three terminal device corresponding to each of the polysilicon dummy structures;using the gate level netlist, performing an automated place and route process to form a layout netlist for fabricating an integrated circuit using the standard cells;laying out the standard cells and laying out routing connections between the standard cells to form a layout for the integrated circuit, using the layout netlist;extracting using a computer from the layout for the integrated circuit a post-layout schematic netlist the post-layout schematic netlist including a three terminal device for each polysilicon dummy structure in the layout netlist, wherein, for each abutted polysilicon dummy structure in the layout, providing in the post-layout schematic netlist a pair of three terminal devices each having a gate terminal that is coupled one to another and each having a body terminal that is coupled one to another; andcomparing the pre-layout schematic netlist to the post-layout schematic netlist.
H04L	9088621	2015-07-21	A method of providing a presence service in a communications system comprising a first user terminal configured to receive presence status information on a second user terminal, the method comprising:displaying, on the first user terminal, presence status information on the second user terminal and a related command element in response to receiving, in the first user terminal from a network apparatus, said presence status information on the second user terminal and a corresponding presence status information identifier;transmitting a presence comment and said presence status information identifier from the first user terminal to said network apparatus in response to recognizing an act performed by a user of the first user terminal, said act comprising the user entering said presence comment via said command element;checking, in said network apparatus, from a presentity authorization list to determine whether the sender of said presence comment is a friend of the presentity;if the sender of said presence comment is determined to be a friend of the presentity, checking, in said network apparatus, to determine whether there is a note element having a timestamp that matches the timestamp of said presence comment; andif it is determined that there is a note element having a matching timestamp, adding, in said network apparatus, said presence comment to said note element.
A61B	9192380	2015-11-24	A method of releasing a surgical buttress from a surgical stapling apparatus, the method comprising the steps of:providing a tool assembly including a cartridge assembly and an anvil assembly defining a longitudinal axis, at least one of the cartridge assembly and the anvil assembly being movable in relation to the other of the cartridge assembly and the anvil assembly;providing a surgical buttress releasably secured to a tissue contacting surface of the anvil assembly, the surgical buttress secured to the anvil assembly by a first connecting member extending transversely across the surgical buttress, the anvil assembly configured to receive a portion of the first connecting member;providing a drive assembly slidably disposed through the tool assembly;providing a release mechanism operatively engaged with the drive assembly and configured to engage the first connecting member;moving the drive assembly along the longitudinal axis; andcontacting the release mechanism with the drive assembly such that the release mechanism severs the first connecting member by an actuation of the release mechanism along the longitudinal axis to release the surgical buttress from the anvil assembly.
A41B	9009866	2015-04-21	An apparatus comprising:a band of flexible material comprising a substantially rectangular body having first and second opposing edges, first and second opposing ends, a front side, and a back side;a portion of the body comprising a covering region, the covering region concealing a garment cuff in an engaged position;a portion of the body comprising a tucking region, the tucking region tucking underneath the garment cuff in an engaged position, wherein the covering region and the tucking region are separated by a fold, the covering region between the first opposing edge and the fold and the tucking region between the second opposing edge and the fold; anda coupling element securing the body to the garment cuff in an engaged position, the coupling element comprising a cuff link, at least one opening disposed along each of the first and second opposing ends of the covering region, and at least two openings disposed along each of the first and second opposing ends of the tucking region, wherein an opening on the first opposing end of the tucking region corresponds with an opening on the second opposing end of the tucking region such that the first and second opposing ends of the covering region and the tucking region are coupled together using one of the at least two openings on the first and second opposing ends of the tucking region with the cuff link, wherein corresponding openings of the at least two openings on the first and second opposing ends of the tucking region are spaced along the first and second opposing ends of the tucking region a different distance from the second opposing edge such that aligning a first pair of corresponding openings of the two or more openings on the first and second opposing ends of the tucking region creates a different length of the covering region and a different location of the fold than aligning a second pair of corresponding openings of the two or more opening on the first and second opposing ends of the tucking region.
F24J,H01L,H02S,Y02B,Y02E	8946542	2015-02-03	A pan structure for bonding solar modules installed on a target position, the pan structure comprising:a plate member having a length and a width, the plate member being configured to have the width to fit a spacing between two rail structures;a pair of edge members configured to respectively attach to two length sides of the plate member, each of the pair of edge members having a first ledge and a second ledge, the first ledge being electrically conductive and having a bottom side being supported by one of the two rail structures and a top side for supporting at least a frame edge region of a solar module, the second ledge connecting the first ledge to the length side of the plate member to provide a distance between the bottom side of the first ledge and the plate member; anda plurality of contact elements spatially distributed along the first ledge, each contact element comprising one or more protrusions on both the top side and the bottom side of the first ledge for generating a bonding force between the frame edge region of the solar module mounted on the first ledge and the rail structures.
H04N	8964044	2015-02-24	An image processing apparatus comprising:an auto continuous-capturing unit for continuously capturing a plurality of unit images;a feature point detecting unit for extracting and setting a feature point to be compared from each unit image;an image matching unit for reducing an error between the unit images by moving and modifying the unit image based on declination of the feature points;an image synthesizing unit for outputting a synthesized image by superposing the unit images;a motion detection range calculating unit for adjusting a size of a motion detection range for detecting the feature point by using a hand-shake limit pixel number, exposure time and capturing interval, wherein the hand-shake limit pixel number is the maximum number of pixels allowing shake not to be substantially perceived in the unit image; anda motion detecting unit for detecting a feature point corresponding to the feature point of the unit image from another unit image in the motion detection range.
H01L	9117928	2015-08-25	A method of forming an array of memory cells, comprising:forming a plurality of pillars, wherein individual pillars have a semiconductor post formed of a semiconductor material and a sacrificial cap on the semiconductor post;forming source regions between columns of the pillars;forming a plurality of gate lines, wherein individual gate lines extend along a corresponding column of pillars and are spaced apart from corresponding source regions, and wherein each gate line surrounds a portion of the semiconductor posts along a corresponding column of pillars;forming self-aligned openings that expose a top portion of corresponding semiconductor posts by selectively removing the sacrificial caps without forming a mask having a pattern of openings corresponding to the pattern of the self-aligned openings; andforming individual drain contacts in the self-aligned openings that are electrically connected to corresponding semiconductor posts.
G05B,H04L	9178760	2015-11-03	An apparatus for data processing comprising:an engineering system device that exchanges data with at least one further node device to which the apparatus, constituting the first node device, is connectable via a PROFINET network, wherein:a control unit that configures the at least one further node device with regard to exchanging data via the PROFINET network in accordance with a desired state,the desired state comprises a desired topology, the desired topology being generated by the control unit with regard to a set of configuration-related information for the particular variant of a modular system or machine, and which enables an actual comparison of the desired and an actual state and therefore the detection of errors,wherein the set of configuration-related information includes specific information that describes each node device in detail, and wherein the specific information includes at least one of:(i) a unique identifier of each node device,(ii) data pertaining to device type and manufacturer of each node device,(iii) a functional description of each node device,(iv) a definition of process data of each node device, and(v) a unique designation of each network interface and respective network interface type of each node device;the control unit generates in response to at least one item of configuration-related information specified by an operator, a set of configuration data which are necessary for configuring the at least one further node device and the control unit transmits at least a subset of this generated set of configuration data to the at least one further node device via the PROFINET network,said control unit is programmed to configure the at least one further node device with regard to physical aspects of the data exchange via the PROFINET network,the set of configuration data generated and required for configuring the at least one further node device with regard to the physical aspects includes the desired topology for the PROFINET network; andthe control unit checks the generated desired topology with regard to the present actual topology; anda display that presents the generated desired topology with regard to the present actual topology.
G02B	8942524	2015-01-27	An optical coupling lens comprising:a body portion comprising a first surface, a second surface perpendicular to the first surface, and a reflecting surface obliquely interconnected between the first surface and the second surface, the first surface defining a plurality of light-passing recess;two locating posts perpendicularly extending from the first surface and being apart from the light-passing recesses;a plurality of converging portions formed on the second surface and corresponding to the light-passing recesses, the reflecting surface configured for reflecting parallel light beams from each of the light-passing recesses toward the corresponding converging portion and for reflecting parallel light beams from each of the converging portions toward the corresponding light-passing recess.
B60L,G07C,Y02T	8981715	2015-03-17	A charging cable for an electric vehicle through which an electric storage device, provided in the vehicle and used for driving the vehicle, is charged from an external power supply, the charging cable comprising:a first coupler that connects the charging cable to a vehicle inlet provided in the vehicle;a second coupler that connects the charging cable to the external power supply;a control device configured to control a usage history of the charging cable, wherein the control device includes: a storage section that stores the usage history of the charging cable; a detection section that detects a usage state of the charging cable and that updates the usage history based on the detected usage state; a determination section that determines whether a service life limit of the charging cable has been exceeded based on the stored usage history and a reference value; andan electric wire section that is provided between the first coupler and the control device and between the second coupler and the control device, through which electric power and a control signal pass,wherein the control device further includes a command output section that sends commands to limit the charging of the vehicle when the determination section determines that the service life limit of the charging cable has been exceeded,the control device is configured to set the control signal to an oscillating signal upon detecting that the first coupler and the second coupler are connected to the vehicle inlet and the external power supply respectively, andthe commands sent by the command output section include a command to stop oscillation of the control signal in response to a determination by the determination section that the service life limit of the charging cable has been exceeded.
H01L	9064803	2015-06-23	A method comprising:providing a memory gate stack over a substrate; providing a word gate over the substrate;and providing a dielectric spacer between the word gate and the memory gate stack, the dielectric spacer comprising a low-k layer and a first high-k layer each layer being formed on the substrate to a same height over the entire length of the low-k and first high-k layers, the dielectric spacer occupying the entire space between the memory gate stack and the word gate.
G06F	9189246	2015-11-17	A method for booting partitions in a processing system, the method comprising:during initialization of the processing system:creating a first basic input/output system (BIOS) region for a main partition of the processing system using a BIOS loader;creating a second BIOS region for a sequestered partition of the processing system using the BIOS loader;booting an operating system in the main partition, the main partition running on one or more first processing units of the processing system;booting the sequestered partition using software stored in a sequestered area of memory of the processing system, wherein the sequestered area of memory is hidden from the main partition, the sequestered partition running on one or more second processing units of the processing system, and wherein the sequestered partition is hidden from the operating system of the main partition; andcontrolling, by the sequestered partition, a device of the processing system by accessing memory of the main partition, wherein the device of the processing system is viewable by the main partition.
G06F,G06Q	9026545	2015-05-05	A method performed by a computer system that comprises one or more processors and memory, comprising:detecting information specifying a domain name for one or more resources;searching a cache for cached content that is associated with the domain name for the one or more resources, with the cached content differing from the information specifying the domain name, and with the cache storing copies of content in the one or more resources obtained through periodic crawling of a network for the content in the one or more resources;retrieving, from the cache, the cached content associated with the domain name;determining a type of language used in the retrieved cached content that is associated with the domain name; andselecting, at least partly based on the determined type of language, an advertisement item for display in at least one of the one or more resources associated with the domain name.
H04B	8971712	2015-03-03	A method for providing carrier-embedded optical radio-over-fiber (RoF) communications, the method comprising:producing a radio frequency (RF) modulated optical signal that is associated with a first optical source and modulated with an electronic baseband;producing an intermediate frequency (IF) signal;producing a RF carrier signal;transmitting, by a first optical fiber, the RF modulated optical signal from the RoF Gateway Router to a remote access point;detecting, at the remote access point, the transmitted RF modulated optical signal to produce an electrical remote access point signal, wherein the remote access point signal is represented by (&#xbd;)cos(2&#x3c9;0t)+(&#x3b1;/2)Re{x(t)exp(j(2&#x3c9;0+&#x3c9;IF)t)}, wherein where x(t) is the baseband signal, &#x3c9;0 is the angular frequency of the RF carrier signal, &#x3c9;IF is the angular frequency of the IF signal, and &#x3b1; is an amplitude factor;filtering the remote access point signal to pass a harmonic of the RF carrier signal to an uplink section of the remote access point;receiving a RF uplink signal;mixing the harmonic of the RF carrier signal with the received RF uplink signal to down-convert the received RF uplink signal to an IF uplink signal;modulating a second optical source or light from the second optical source with the IF uplink signal to produce an IF uplink optical signal;transmitting, by a second optical fiber, the IF uplink optical signal from the remote access point to the RoF Gateway Router; anddetecting, at the RoF Gateway Router, the IF uplink optical signal.
A61B,B21K	9072559	2015-07-07	An adjustable length bone screw for use in a surgical operation where the screw is inserted into a bone, the bone screw comprising:a screw head extending between a proximal end and a distal end and having an outer surface, and a shaft, the screw head and the shaft extending along a longitudinal axis, the shaft having a shaft outer surface and extending between the distal end of the screw head and a distal tip, wherein the outer surface of the screw head has a maximum diameter measured along a first direction that is perpendicular to the longitudinal axis that is greater than a maximum diameter of the shaft outer surface measured along the first direction;a plurality of threaded portions located at discrete positions longitudinally along the shaft, and a plurality of unthreaded portions located at discrete positions longitudinally along the shaft, wherein the threaded portions are separated longitudinally by the unthreaded portions, and wherein the threaded portions contain a series of threads extending at least partially circumferentially around the shaft;wherein the plurality of the threaded portions each have at least two cutting flutes each defined by a notch formed radially into the screw shaft and extending longitudinally along the length of the threaded portions such that the threads of the threaded portions are formed into at least two discontinuous thread segments extending circumferentially around the shaft, andwherein a plurality of the unthreaded portions are tapered along the shaft longitudinally in a proximal to distal direction and a plurality of the threaded portions proximally adjacent to the plurality of tapered unthreaded portions contain at least one thread that is tapered in the same angle as the taper for the adjacent unthreaded portion.
G06T	9047653	2015-06-02	A method of blending first and second portions of a stitched digital image of a document, the method comprising the steps of:using a computer, detecting respective overlapping or adjacent boundaries of the first and second portions;using a computer, categorizing pixels on each boundary of the first and second portions as either a background pixel or a foreground pixel; andusing a computer, modifying a pixel of the first or second portion based on a pixel value difference between corresponding background pixels on the respective boundary of the first and second portions, wherein the corresponding background pixels on the respective boundary of the first and second portions are those closest in distance to the pixel being modified.
B81C,C23C,H01L	8946864	2015-02-03	An ion-implanted structure comprising:a substrate comprising a first material;an embedded structure comprising a second material embedded within the first material of the substrate; anda substantially continuous film comprising metal implanted within the first material of the substrate, disposed within the second material of the embedded structure or immediately adjacent to and contacting the second material of the embedded structure.
H03H	9166555	2015-10-20	A phase shifter, comprising:a bulk acoustic wave resonator (BAWR) comprising a first connector and a second connector, wherein the first connector of the BAWR is configured to be directly connected to a first port;a first capacitor comprising a first connector and a second connector, wherein the first connector of the first capacitor is configured to be connected to ground,; and the second connector of the first capacitor is configured to be connected to the first connector of the BAWR;a second capacitor comprising a first connector and a second connector, wherein the first connector of the second capacitor is configured to be connected to ground, andwherein the second connector of the BAWR, the second connector of the second capacitor, and a second port are connected.
C23C,H01J,H01L	8931432	2015-01-13	A vacuum processing apparatus comprising:a plurality of plate-like discharge electrodes that is supplied with high-frequency power from a power supply unit through feeding points thereof, and forms plasma with respect to a substrate placed on a counter electrode respectively, anda plurality of matching boxes,wherein one end of each of the plurality of discharge electrodes is electrically connected to another end of each of the plurality of discharge electrodes via a loop circuit,wherein the plurality of matching boxes tunes phases and amplitude of the high-frequency power supplied to the plurality of discharge electrodes at the feeding points respectively,wherein the plurality of matching boxes includes one matching box which is configured to tune the high-frequency power supplied to one discharge electrode, the one matching box corresponding to the one discharge electrode,wherein the one matching box is configured to set an impedance value such that a reflected power returned from the one discharge electrode to the power supply unit is approximately minimized,wherein impedance values of the plurality of matching boxes are set to approximately the same value as a measured value obtained by measuring the impedance of the one matching box, andwherein the vacuum processing apparatus is configured to performreflected-power tuning, in which among the plurality of matching boxes that tunes phases and amplitudes of the high-frequency power supplied to the plurality of discharge electrodes that form plasma with respect to the substrate placed on the counter electrode respectively, impedance of the one matching box, which tunes the high-frequency power supplied to the one discharge electrode, is set such that reflected power returned from the one discharge electrode is approximately minimized, andsetting, in which the impedance of other matching boxes is set to approximately the same impedance as the impedance of the one matching box.
E04B,E04H,F16F	8973312	2015-03-10	A self-centering damper comprising:an elongated damping body havingtwo sides;two end segments;an I-shaped cross-section;a first wing plate having:two sides, each of the two sides having a width;a middle, having:a width narrower than the width of each of the two sides of the first wing plate; andtwo opposite sides; andtwo concave segments respectively formed on the opposite sides of the middle of the first wing plate;a second wing plate parallel to the first wing plate, the second wing plate having:two sides, each of the two sides having a width;a middle, having:a width narrower than the width of each of the two sides of the second wing plate; andtwo opposite sides; andtwo concave segments respectively formed on the opposite sides of the middle of the second wing plate; anda supporting element connected to the first wing plate and the second wing plate; andat least one returning device connected to one of the sides of the damping body and each one of the at least one returning device havingtwo holding rib plates respectively connected to the end segments of the damping body;a supporting rib plate securely connected to the damping body between the holding rib plates; andat least one pre-stressed stay wire extending through the supporting rib plate and connected to the holding rib plates in a tension condition.
G01J,G02B	9063325	2015-06-23	A tunable interference filter comprising:a first substrate;a second substrate opposed to the first substrate;a first reflection film provided on a first reflection film fixing surface of the first substrate facing the second substrate;a second reflection film provided on a second reflection film fixing surface of the second substrate facing the first substrate and opposing the first reflection film across a gap between the first and second reflection films; anda first electrode provided on a first electrode surface of the first substrate facing the second substrate, the first electrode surrounding the first reflection film fixing surface in a plan view, and being spaced apart from the first reflection film fixing surface in a thickness direction of the first substrate,wherein the first electrode surface is spaced apart from the second substrate by a different distance than the distance from the first reflection film fixing surface to the second substrate,the first reflection film covers a step between the first reflection film fixing surface and the first electrode surface, anda layered part in which the first reflection film and the first electrode are stacked is provided on the first electrode surface, the layered part being spaced apart from the second substrate by a larger distance than the distance from the first reflection film fixing surface to the second substrate.
F16K,H01F,H01J,H02K	9172277	2015-10-27	An actuator comprising:an electrically conductive coil defining a longitudinal axis (L) and having a plurality of winding turns;a magnet spaced from said winding turns in radial direction with reference to said longitudinal axis (L);said coil having a side facing away from said magnet;a first conductive element having a mid region at least partially covering said coil on said side facing away from said magnet;said magnet having a side facing away from said winding turns of said coil;a second conductive element having a mid region at least partially covering said magnet on said side thereof facing away from said winding turns of said coil;said first conductive element projecting beyond said coil in axial direction with reference to said longitudinal axis (L);said second conductive element projecting beyond said magnet in axial direction with reference to said longitudinal axis (L);said first and second conductive elements having respective collar-shaped projections whereat said first and second conductive elements project beyond said coil and said magnet, respectively; and,at least one of said first and second conductive elements being made of soft-magnetic powder composite material.
H03M	9148166	2015-09-29	A successive approximation register analog to digital converter (SAR ADC), configured to receive an input voltage and a set of reference voltages, comprising:a charge sharing DAC comprising an array of MSB (most significant bit) capacitors and an array of LSB (least significant bit) capacitors;a zero crossing detector coupled to the charge sharing DAC, the zero crossing detector configured to generate a digital output;a coarse ADC (analog to digital converter) configured to receive the input voltage and configured to generate a coarse output, wherein a predefined offset is added to a residue of the coarse ADC; anda successive approximation register (SAR) state machine, coupled to the coarse ADC and the zero crossing detector and, configured to generate a plurality of control signals, wherein the plurality of control signals is configured to operate the charge sharing DAC in a sampling mode, an error-correction mode and a conversion mode.
G06F	9019226	2015-04-28	A method for detecting an object on a capacitance sensing array, the method comprising:in a first mode:configuring a capacitance sensing circuit to measure self capacitance,measuring a first self capacitance of a first electrode,measuring a second self capacitance of a second electrode,detecting, based on at least one of the first self capacitance of the second self capacitance, the object being proximate to a surface intermediate to the object and the first and second electrodes;responsive to detecting, configuring the capacitance sensing circuit to measure mutual capacitance of at least the first electrode and the second electrode;transmitting a transmit (TX) signal on the first electrode;receiving a receive (RX) signal on the second electrode, the RX signal responsive to a mutual capacitance between the first and second electrode; andcalculating a position of the object proximate to, but not contacting, the substrate; andin a second mode:configuring the capacitance sensing circuit to measure mutual capacitance;transmitting a transmit (TX) signal on the first electrode;receiving a receive (RX) signal on the second electrode, the RX signal responsive to a mutual capacitance between the first and second electrode; andcalculating a position of the object in physical contact with the substrate intermediate to the object and the first and second electrodes.
H01L,H03K	8928893	2015-01-06	A proximity sensor comprising:a sensor package comprising a lens module; a radiation source and a radiation detector housed within the sensor package, the lens module of the sensor package being positioned over the source and the detector; and an optical isolator adapted to substantially prevent internal propagation of radiation between the source and the detector within the sensor package,the optical isolator comprising:a baffle positioned within the sensor package between the source and the detector, said baffle comprising opposed side surfaces and an upper edge extending between the opposed side surfaces; anda groove formed in an upper surface of the lens module, a bottom-most part of the groove being positioned adjacent to said upper edge of the baffle.
G11C	9025388	2015-05-05	A memory device, comprising:an array of memory cells; andcontrol circuitry coupled to the array of memory cells, and configured to:apply programming pulses to a memory cell until a threshold voltage (Vt) of the memory cell is greater than or equal to a pre-program verify voltage level and less than a program verify voltage level; andadjust a programming pulse step voltage magnitude for a later programming pulse applied to the memory cell, wherein the programming pulse step voltage magnitude for the later programming pulse is less than a programming pulse step voltage magnitude used to arrive at a preceding programming pulse.
H04B,H04L,H04W	9088939	2015-07-21	A method, comprising:receiving, at a cell relay, an association of an evolved packet system (EPS) bearer to a local radio bearer and a portion of an identifier related to the EPS bearer from an upstream evolved Node B (eNB);storing, at the cell relay, the association and the portion of the identifier with an index of the local radio bearer in a routing table; andreceiving, at the cell relay, data related to the EPS bearer from the upstream eNB over the local radio bearer.
F04B,F04C,F04D	9175691	2015-11-03	A method of producing a compressed gas and controlling flow rate of the compressed gas, said method comprising:compressing a gas within a compression system having an electric motor and at least one stage of compression directly coupled to the electric motor and configured to compress a gas and thereby to produce the compressed gas, the compression system having a restricted speed zone corresponding to a range of speeds of the electric motor at which vibrational modes can damage the compression system;determining a flow rate and a desired flow rate of the compressed gas through the compression system;controlling the flow rate of the compressed gas through the compression system to produce the desired flow rate by adjusting inlet guide vanes of a centrifugal compressor of the at least one stage of compression and by adjusting the speed of the electric motor and therefore, an impeller of the centrifugal compressor with a speed controller;wherein, when the desired flow rate is able to be obtained with the speed of the electric motor situated above or below the restricted speed zone, the flow rate of the compressed gas through the compression system is controlled by adjusting the speed of the electric motor with the speed controller and with the inlet guide vanes set at a 0&#xb0; swirl angle; andwherein, when the desired flow rate is not able to be obtained without the speed of the electric motor situated within the restricted speed zone, the flow rate of the compressed gas through the compression system is controlled by setting the speed of the electric motor with the speed controller at an upper speed, above the restricted speed zone and adjusting the inlet guide vanes to obtain the desired flow rate;wherein, when the speed of the electric motor is set at the upper speed, above the restricted speed zone and the desired flow rate requires adjusting the speed of the electric motor to a speed below the restricted speed zone, the speed of the electric motor is decreased in decreasing speed steps until a lower speed, below the restricted speed zone, is obtained and the inlet guide vanes are successively opened during each of said decreasing speed steps such that the inlet guide vanes are set at the 0&#xb0; swirl angle when the speed of the electric motor is at the lower speed, below the restricted speed zone; andwherein, when the speed of the electric motor is set below the restricted speed zone and the desired flow rate requires adjusting the speed of the electric motor to a new speed above the restricted speed zone, the speed of the electric motor is increased in increasing speed steps until the new speed, above the restricted speed zone, is obtained and the inlet guide vanes are successively closed during each of said increasing speed steps such that the position of the inlet guide vanes required to obtain the desired flow rate will be reached when the new speed, above the restricted speed zone is reached.
H03H	9077316	2015-07-07	A method for estimating finite impulse response (FIR) of a FIR filter, the method comprising:by at least one controller,injecting specified FIR tap values into a first captured waveform that results from transmitting a raw waveform through a transmitter circuit including the FIR filter having pre and post cursor tap values set to zero to create an expected waveform,injecting the specified FIR tap values into the raw waveform to create an ideal waveform,projecting the expected and ideal waveforms onto a second captured waveform that results from transmitting the ideal waveform through the transmitter circuit with the pre and post cursor tap values set to the specified FIR tap values to create a compensated waveform, andextracting FIR tap values from the compensated waveform.
H01L	9059144	2015-06-16	A method for forming a molded die assembly, comprising:attaching a first major surface of a semiconductor die onto a package substrate;attaching a heat spreader to a second major surface of the semiconductor die, whereinthe second major surface is opposite the first major surface,the heat spreader includes a pedestal portion that contacts the semiconductor die and side extensions that are raised above the pedestal and the semiconductor die when the heat spreader is mounted above the semiconductor die, andthe semiconductor die, package substrate, and heat spreader form a die assembly;conforming a die release film to a transfer mold;closing the transfer mold around the die assembly such that the die release film is compressed against the heat spreader and a cavity is formed around the die assembly;transferring a thermoset material into the cavity; andreleasing the die assembly from the die release film and the transfer mold, wherein a top surface of the thermoset material is substantially flush with a majority of a top surface of the side extensions of the heat spreader.
B01J,C07C,Y02P	9206107	2015-12-08	A method for producing a carboxylic acid and an alcohol, the method comprising:producing a carboxylic acid and an alcohol by hydrolyzing a carboxylic acid ester in water in the presence of an ammonium pyrosulfate catalyst represented by the following general formula (1):wherein, R1 and R4 each independently represent an aryl group, and R2, R3, R5, and R6 each independently represent an aryl group, an alkyl group, an alkenyl group, an alkynyl group, a cycloalkyl group, or a hydrogen atom.
H04L	9118592	2015-08-25	A device comprising:a communication interface; androuting logic in communication with the communication interface, the routing logic configured to:receive, through the communication interface, status information advertised by a remote routing device periodically, aperiodically, by request, based on an event occurrence, or any combination thereof, wherein the status information comprises an indication of the remote routing device's current capability to handle network traffic;receive network data through the communication interface; androute the network data in view of the status information.
A61B,G06T	9208582	2015-12-08	An image analysis method performed by an image processing apparatus to analyze medical-purpose image data corresponding to an image of an object's organ having an anatomically symmetric shape, wherein said image analysis method performed by said image processing apparatus comprises:(1A) a step of inputting the medical-purpose image data corresponding to the image of the organ;(1B) a step of generating mask image data which collectively express a mask image corresponding to a region of the organ, based on processing the image data corresponding to the image of the organ;(1) a step of determining, based on the image data, and designating, a centerline of the organ in the image constituted by the image data;(2) a step of setting N-1 parting lines based on processing of the mask image data and the centerline designated in the step (1),wherein N is an integer greater than two, the N-1 parting lines intersect at a barycenter of the mask image corresponding to a region of the organ and form an angle &#x3b8;&#xb7;n with the centerline, wherein &#x3b8; is an angle calculated by dividing 180&#xb0; by N, and n denotes an integer ranging from 0 to N-1;(3) a step of setting automatically a plurality, or more than one pair, of regions of interest at symmetrical opposite positions, based on processing of the mask image data obtained in step (1B) and the parting lines set in the step (2);(4) a step of generating an analytic value of the image data in each of the regions of interest, based on processing of the image data in each of the regions of interest set in the step (3); and(5) a step of assessing the state of the object according to a degree of a difference in the analytic value between the opposite regions of interest.
C12N	9012426	2015-04-21	A method of treating a gastric cancer by promoting expression of PTEN in hypoxia conditions in a subject in need thereof, the method comprising administering to the subject a composition comprising a pharmaceutically effective amount of PNA (peptide nucleic acid) complementary to microRNA-382 as a microRNA-382 inhibitor.
A01B,A01D,G01C,G05D	8983767	2015-03-17	A computer-implemented method for generating a swath coverage pattern for a given working area to be traversed by a vehicle or equipment attached thereto, the method comprising steps, each implemented in a computer system, of:(a) designating a covered area denoted by a polygon defined by an ordered set of vertices; and(b) partitioning the covered area into an ordered set of swaths by:(i) defining a headlands swath pattern for providing a continuous turnaround buffer for the vehicle or equipment attached thereto, said headlands swath pattern forming a border around an inner perimeter of the covered area; and(ii) defining a series of parallel swaths to cover an interior portion of the covered area within the headlands swaths, said parallel swaths alternating back and forth across the interior portion of the covered area.
B29C,B29K,B29L,B60K,F02M,Y10T	9181902	2015-11-10	A fuel tank comprising:a fuel tank body which is constituted by an upper shell and a lower shell, upper shell has an upper plate portion having an inner surface;a fuel supply device which is installed in the fuel tank body and pressure-feeds fuel in the fuel tank body to an internal combustion engine of a vehicle; anda fuel vapor treating apparatus mounted to the upper shell and configured to enable adsorption of fuel vapor;wherein the fuel supply device includes a mounting mechanism configured to mount the fuel supply device to the upper plate portion, the mounting mechanism including a ceiling plate portion that engages the inner surface of the upper plate portion of the upper shell;wherein a pipe and a wire of the fuel supply device extend through the ceiling plate portion; andwherein the pipe and the wire of the fuel supply device and a pipe and a wire of the fuel vapor treating apparatus are configured to penetrate a wall of the upper shell.
C03C,C08F,C09D,H01L,Y02E	9150750	2015-10-06	A method of forming a coating on a glass, metal, or ceramic substrate comprising:(a) applying to a substrate to a thickness of at least 50 microns a low VOC, heat-curable medium comprising:(i) 20-95 wt % of heat curable mono-functional acrylate monomers, di-functional acrylate monomers and tri-functional acrylate monomers selected from the group consisting ofa. monofunctional acrylate monomers selected from the group consisting of 2(2-ethoxyethoxy) ethyl acrylate; 2-phenoxyethyl acrylate; isodecyl acrylate; lauryl acrylate; and stearyl acrylate;b. difunctional acrylate monomers including at least one of a C2-C22 aliphatic group and a C6-C30 aromatic group, andc. trifunctional acrylate monomers including at least one of a C2-C24 aliphatic group and a C6-C36 aromatic groupwith the proviso that the heat-curable medium includes at least one monofunctional acrylate monomer;(ii) 0.1-20 wt % of a solvent having a boiling point above 150&#xb0; C. at STP,(iii) 0.1-20 wt % of an acrylic resin, and(iv) 0.1-18 wt % of an oligomer including a polyester residue and a urethane residue wherein the medium has a viscosity of 40,000 to 150,000 cps when measured at a shear rate of 0.125/second, and(b) curing the medium to form a cross linked network.
A45C,G01N,G02B,G08G	9091808	2015-07-28	An assembly for performing traffic sign inspections, comprisinga first enclosure for removably housing a first plurality of sheetings including:a first white sheeting having a retroreflectivity level of at least 35 cd/lx/m2;a second white sheeting having a retroreflectivity level of at least 50 cd/lx/m2;a third white sheeting having a retroreflectivity level of at least 120 cd/lx/m2;a fourth white sheeting having a retroreflectivity level of at least 250 cd/lx/m2;a first green sheeting having a retroreflectivity level of at least 7 cd/lx/m2;a second green sheeting having a retroreflectivity level of at least 15 cd/lx/m2;a third green sheeting having a retroreflectivity level of at least 25 cd/lx/m2;a first yellow sheeting having a retroreflectivity level of at least 50 cd/lx/m2;a second yellow sheeting having a retroreflectivity level of at least 75 cd/lx/m2;a first orange sheeting having a retroreflectivity level of at least 50 cd/lx/m2;a second orange sheeting having a retroreflectivity level of at least 75 cd/lx/m2;a first red sheeting having a retroreflectivity level of at least 7 cd/lx/m2;an attachment device for attaching one of said first plurality of sheetings to a traffic sign; anda first handheld light source.
F16J,F16K,G02C	9122072	2015-09-01	A sealing device comprising:a guide member having at least one internal cavity and a passageway intersecting the internal cavity; anda pinch member slidably engaged within the internal cavity of the guide member,wherein a portion of the pinch member slides across the passageway and into a portion of the internal cavity to deform and seal a fluid tube extending through the passageway;wherein at least one extension is defined on one of the pinch member and the internal cavity, and a corresponding recess is defined in one of the other of the pinch member and the internal cavity, the extension engaging the recess to secure the pinch member and to inhibit proximal travel, andwherein a gap is defined between a wall of the internal cavity and a portion of the pinch member extending therein, the gap being less than approximately two times a wall thickness of the fluid tube.
C12Q,G01N	8940482	2015-01-27	A cell based assay for assessing the activity of a Botulinum Neurotoxin (BoNT), comprising:measuring the disappearance of a single fluorescent or luminescent label located at the C-terminus end of a BoNT substrate in a recombinant cell in the presence of a BoNT,wherein the recombinant cell expresses a BoNT substrate, which substrate comprises said single fluorescent or luminescent label at its C-terminus,wherein said BoNT substrate generates a N-terminus fragment and a C-terminus fragment upon cleavage by the BoNT and where the C-terminus fragment and label are degraded by the recombinant cell's ubiquitin proteasome pathway;wherein the substrate is not labeled as a FRET pair.
A01G,Y02P	9078404	2015-07-14	Agricultural cultivating equipment adapted for cultivating crops, said agricultural cultivating equipment comprising:a supporting unit;a rotating unit disposed rotatably on said supporting unit and rotatable about a vertical axis, said rotating unit having an inner annular portion, an outer annular portion disposed around and connected to said inner annular portion, an operation region defined in said inner annular portion, and a cultivation region defined between said inner and outer annular portions;a driving unit connected to said rotating unit for driving rotation of said rotating unit relative to said supporting unit;a loading unit including a plurality of loading plates disposed on said rotating unit and in said cultivation region and spaced vertically apart from each other, and a plurality of loading tray assemblies disposed respectively on said loading plates and each including a plurality of loading trays adapted to permit the crops to be planted therein, said loading plates being co-rotatable with said rotating unit;an operating unit disposed on said supporting unit and in said operation region of said rotating unit and adapted to permit a user to move from said supporting unit onto a selected one of said loading plates therethrough for performing a cultivating operation on the crops;a supply unit including a water supplying device disposed on said rotating unit for spraying water into said loading trays of said loading unit, and a plurality of lighting members disposed on said rotating unit for lighting said loading trays of said loading unit; anda control unit electrically connected to said rotating unit, said driving unit, said loading unit, said operating unit, and said supply unit and operable for controlling operation of said rotating unit, said driving unit, said loading unit, said operating unit, and said supply unit.
G06F,H04L	8958548	2015-02-17	A method, comprising:selecting a non-prime multiplier (R) in accordance with a formula R=P1wherein P is a prime number and I a number is selected such that PI is less than the number of bits in every packet of a plurality of packets, the plurality of packets corresponding to a message;generating a non-prime modulus (M) wherein M is generated in accordance a formula M=(PN*QT)&#x2212;1, where N, Q and T are numbers selected such that M is greater than the number of bits in every packet of the plurality of packets and N&#x3e;1, wherein M and R are relative prime numbers;employing M and R, in accordance with a cryptography system, to encrypt each packet of plurality of packets to generate a plurality of encrypted packets;aggregating the plurality of encrypted packets to generate an encrypted message; andtransmitting the encrypted message.
H04B	9197278	2015-11-24	A communications system comprising:a plurality of subsystems;a reference oscillator coupled to the plurality of subsystems, wherein the reference oscillator is configured to provide a reference signal to the plurality of subsystems; anda controller coupled to the reference oscillator and the plurality of subsystems, wherein the controller is configured to:receive, from a first subsystem in the plurality of subsystems, a request to change a characteristic of the reference oscillator,determine, based on a priority of the request, whether to process the request,send, in response to determining that the request should be processed, a notification to the plurality of subsystems of an upcoming change to the characteristic of the reference oscillator,initiate, in response to determining that the request should be processed, a reconfiguration of the reference oscillator to meet a requirement of the first subsystem,determine whether performance of one or more subsystems in the plurality of subsystems will be impaired by the reconfiguration of the reference oscillator, andmitigate an impact on the one more subsystems in the plurality of subsystems by initiating a frequency tuning of respective one or more internal oscillators of the one or more subsystems in the plurality of subsystems.
B62D,E01C,E02D	9016710	2015-04-28	An apparatus for the articulated connection of a front vehicle frame and a rear frame of a construction machine, comprising:a pivot joint having a vertical pivot axis (K) on one of the front and rear vehicle frames;a crab-steering joint located at a distance from the pivot joint having a vertical rotational axis (H) on another of the front and rear vehicle frames;a connection support connecting the pivot joint and the crab-steering joint with one another, wherein the connection support is mounted as a pivoting arm on the front vehicle frame on one end and on the rear vehicle frame on the other end, and wherein the connection support is part of the crab-steering joint and carries the pivot joint;a steering drive for relative pivoting of the front frame and the rear frame about the pivot axis (K), with the steering drive being linked on one end to the vehicle frame which is connected to the pivot joint and on the other end to the connection support;a crab-steering drive for setting a lateral offset (s) between the front frame and the rear frame, the crab-steering drive being linked on one end to the vehicle frame that is near the pivot joint and on the other end to the connection support;wherein the rotational axis (H) of the crab-steering joint is connected more closely to the vehicle frame which is linked to the crab-steering joint then to the pivot axis (K) such that a distance between the front and rear vehicle frames increases when the crab-steering drive is actuated; andwherein a maximum increase in length of the construction machine (&#x394;L) achievable by actuating the crab-steering drive is equal the length (l) of the connection support.
B29D,G02B	9097823	2015-08-04	A lens array comprising:a plate-shaped light-absorbing material that is provided with a plurality of holes separated from each other; anda lens formed of a polymer in each of the holes,wherein the holes are provided in a staggered configuration, andwherein a side surface of each of the holes in the plate-shaped light-absorbing material is subjected to a liquid-repellent treatment, andwherein the polymer has a vertical convex shape on the side surface of each respective hole, thereby allowing for a curvature of the lens or a length of the lens to be adjusted by an amount of liquid.
B01J,C07C	9073042	2015-07-07	A process for hydrogenating feedstock comprising acetic acid in the presence of hydrogen to product comprising ethanol in a reaction zone under hydrogenation conditions over a catalyst composition promoted with cobalt, rhodium, cesium or a combination thereof on a previously calcined composition comprising at least one Group VIII metal and tin on a support material.
H01L	9136164	2015-09-15	A method for forming a semiconductor device, comprising:providing a substrate comprising a first region and a second region;providing first metal layers in the first region and the second region of the substrate, wherein the first metal layers have a top surface coplanar with a top surface of the substrate;forming an interlayer dielectric layer over the substrate, wherein the interlayer dielectric layer comprises a trench in the first region, and the interlayer dielectric layer in the second region has a top surface coplanar with a bottom of the trench in the first region;forming through-holes in the interlayer dielectric layer, each through-hole corresponding to one first metal layer in the first region and the second region;forming a polymer layer to fill the through-holes and the trench in the interlayer dielectric layer and to cover the top surface of the interlayer dielectric layer in the first and second regions, wherein the polymer layer has a different solubility in an exposed area and in a non-exposed area;exposing and developing the polymer layer to form vias, each including an upper via in the polymer layer and a lower via in the interlayer dielectric layer, wherein each via is formed on a corresponding first metal layer in the first and second regions;forming a second metal layer to fill each via and to electrically connect to the corresponding first metal layer; andremoving the polymer layer between adjacent second metal layers to form air gaps between the adjacent second metal layers in the second region.
H02P	8994298	2015-03-31	A movement control apparatus, comprising:an actuator that includes a permanent magnet and a driving coil facing the permanent magnet via a predetermined air gap, and causes a driven body connected to the driving coil to perform reciprocating movement;a signal generation unit that generates a velocity command signal which indicates a target velocity of the driven body;a driving unit that supplies electric current corresponding to an inputted driving signal to the driving coil of the actuator;a voltage detection unit that detects induced voltage generated in the driving coil with electric current supplied by the driving unit, and outputs a voltage signal corresponding to the detected induced voltage;a signal correction unit that corrects, based on the driving signal and the voltage signal outputted from the voltage detection unit, the voltage signal to adjust a shift of a resistance value from a predetermined reference resistance value of the driving coil, thereby generating a velocity signal that indicates movement velocity of the driven body; anda control unit that generates the driving signal based on the velocity command signal generated by the signal generation unit and the velocity signal generated by the signal correction unit, and outputs the driving signal to the driving unit, whereinthe signal correction unit includes:a correction signal generation unit which generates a correction signal that corresponds to a shift of the resistance value from the reference resistance value of the driving coil based on (i) an electric current signal which corresponds to the driving signal and (ii) the velocity signal;a multiplication unit that multiplies the correction signal generated by the correction signal generation unit by the electric current signal that corresponds to the driving signal, and outputs a multiplication result obtained by the multiplication; anda velocity signal generation unit that generates the velocity signal from the multiplication result outputted from the multiplication unit and the voltage signal outputted from the voltage detection unit.
C12N,C12Y	9029337	2015-05-12	A compound comprising a single-stranded modified antisense oligonucleotide consisting of 12 to 30 linked nucleosides targeted to a human Factor 7 nucleic acid, wherein the oligonucleotide has a sequence that is least 90% complementary to SEQ ID NO: 1 as measured over the entirety of said oligonucleotide, and wherein the oligonucleotide comprises:a gap segment consisting of linked deoxynucleosides;a 5&#x2032; wing segment consisting of linked nucleosides;a 3&#x2032; wing segment consisting of linked nucleosides;wherein the gap segment is positioned immediately adjacent to and between the 5&#x2032; wing segment and the 3&#x2032; wing segment and wherein each nucleoside of each wing segment comprises a modified sugar.
G06F	8972388	2015-03-03	A method comprising:receiving a sequence of one or more characters entered into a search field on a computing device, wherein the sequence of characters represents a first partial query;providing a first list of query completions for the first partial query for display on the computing device;receiving one or more additional characters entered into the search field upon display of the first list, wherein the sequence of characters and the additional characters cumulatively represent a second partial query;obtaining a second list of query completions for the second partial query;identifying one or more query completions in the second list that appear in the first list;calculating demotion scores for the identified query completions, wherein a demotion score for a particular identified query completion is based at least in part on a weighted value associated with a period of time between when the first list was displayed and when the additional characters were entered, wherein the weighted value is a non-binary measure based on a magnitude of the period of time;using the demotion scores to demote one or more of the identified query completions to a lesser position within the second list, thereby forming a refined second list; andproviding the refined second list of query completions for display on the computing device.
G06F	9098379	2015-08-04	A method comprising:storing virtual appliance image templates at a caching location, based on a historical request pattern;receiving a request for a virtual appliance image; andproviding, to a second location, a return virtual appliance image responsive to the received request;wherein said providing of the return virtual appliance image comprises:determining a difference between the requested virtual appliance image and one or more of the stored virtual appliance image templates;at the caching location, obtaining a supplement which corresponds to the determined difference between the requested virtual appliance image and the one or more of the stored virtual appliance image templates; anddelivering the one or more of the stored virtual appliance image templates and the supplement to the second location; andapplying a cost function for caching at the caching location, wherein the cost function is determined by variables comprising:a frequency variable indicating a frequency of a request of one or more supplements deriving from a third location;a size variable indicating a size of the one or more supplements deriving from a third location;a caching variable indicating whether the one or more supplements are already cached at the caching location; anda supplement variable indicating whether the virtual appliance image is to be created using the one or more supplements;wherein the cost function comprises at least a product of the frequency variable and the size variable.
C07K,C12N,C12P,C12Y	8951762	2015-02-10	A method of increasing production of short-chain fatty acids in a host cell or organism, which method comprises introducing into the host cell or organism and expressing therein a nucleic acid molecule comprising a nucleotide sequence encoding a mutant acyl-acyl carrier protein (ACP) thioesterase (TE) acyl-ACP TE), which is derived from wild-type Bryantella formatexigens acyl-ACP TE having the amino acid sequence of SEQ ID NO:24, comprises two amino acid mutations comprising N169Y and S222I, and has increased thioesterase activity compared to wild-type Bryantella formatexigens acyl-ACP TE having the amino acid sequence of SEQ ID NO:24, whereupon the production of short-chain fatty acids in the host cell or organism is increased and wherein the mutant acyl-ACP TE produces more short-chain fatty acids in the host cell or organism than the corresponding wild-type acyl-ACP TE.
G11C,H01L	9147690	2015-09-29	An erasable programmable single-poly nonvolatile memory, comprising:a substrate structure;a first PMOS transistor comprising a select gate, a first source/drain region, and a second source/drain region, wherein the select gate is connected to a select gate voltage, and the first source/drain region is connected to a source line voltage;a second PMOS transistor comprising the second source/drain region, a third source/drain region, and a floating gate, wherein the third source/drain region is connected to a bit line voltage and the first source/drain region, the second source/drain region, and the third source/drain region are constructed in a N-well region;an erase gate region adjacent to the floating gate, wherein the erase gate region comprises a first n-type region connected to an erase line voltage and a P-well region, wherein the N-well region and the P-well region are formed in the substrate structure; anda first isolation structure and a second isolation structure formed in the substrate structure, wherein the first isolation structure is located between the N-well region and the P-well region, and the erase gate region is located between the first isolation structure and the second isolation structure;wherein a plurality of electrons are transmitted through a channel region of the second PMOS transistor to the floating gate and the electrons are injected into the floating gate in a programmed state; and the electrons are removed from the floating gate and discharged out of the erasable programmable single-poly nonvolatile memory through the first n-type region in an erased state;wherein the P-well region comprises: a first p-type region formed under a surface of the substrate structure and contacted to the first n-type region; a plurality of second p-type regions formed under the first isolation structure and the second isolation structure; and a third p-type region formed under the first p-type region;wherein the first p-type region and the third p-type region are located between the second p-type regions, and both the first p-type region and the third p-type region are directly contacted to the second p-type regions; andwherein a dosage in the second p-type regions is higher than a dosage in the first p-type region, and a dosage in the third p-type region is higher than or equal to the dosage in the first p-type region.
C07K,C12N,G01N	9062309	2015-06-23	Small interfering RNA comprising a nucleotide sequence selected from the group consisting of SEQ ID NO:s 2 to 6, and inhibiting KIAA1524.
H04L,H04M,H04Q,H04W	8929358	2015-01-06	A method, comprising:receiving, from a mobile switching network, by a carrier gateway server deployed in a public cellular network, a call origination request of a call destined to a mobile station associated with an enterprise network;providing, by an enterprise gateway server, a first routing information to the carrier gateway server, the first routing information comprising at least one policy associated with a member of the enterprise network linked to the call origination request and routing instructions based on the at least one policy, wherein the at least one policy comprises enterprise call routing criteria applied to the call comprising at least one of a restriction on times the call may be routed and restrictions on destinations where the call may be routed;transmitting from the carrier gateway server to the mobile switching network, a second routing information different from the first routing information, the second routing information comprising destination information and call path information for routing the call to a destination; androuting, by the mobile switching network, the call to the destination;supporting, by the enterprise gateway server, call routing between a private branch exchange deployed in the enterprise network and the public cellular network;wherein the enterprise gateway server routes to a final destination via the private branch exchange.
H01L	8937381	2015-01-20	A fan out buildup substrate stackable package comprising:an electronic component comprising:an active surface comprising bond pads;sides;an inactive surface; andthrough electronic component vias extending through the electronic component between the active surface and the inactive surface of the electronic component, wherein ends of the through electronic component vias at the active surface define active surface through via terminals;a package body enclosing the electronic component and directly contacting the sides;a first die side buildup dielectric layer applied to the active surface of the electronic component and a first surface of the package body, a second surface of the package body being coplanar with the inactive surface of the electronic component;a first die side circuit pattern in contact with the first die side buildup dielectric layer and electrically connected to the bond pads; andthrough vias extending through the package body and the first die side buildup dielectric layer, the through vias being electrically connected to the first die side circuit pattern.
C22C,C22F,G21C,Y02E	9099205	2015-08-04	A zirconium alloy for a nuclear fuel cladding comprising 1.1 wt % of niobium (Nb); at least two elements selected from the group consisting of tin (Sn), iron (Fe) and chromium (Cr); 0.05 wt % of copper (Cu); 0.12 wt % of oxygen (O); 0.01 wt % of silicon (Si) and a remaining amount of zirconium (Zr), in which, the amount of tin (Sn) being 0.2 wt %, the amount of iron (Fe) being 0.5 wt %, and the amount of chromium (Cr) being 0.1 to 0.2 wt %.
H04B	9191124	2015-11-17	An opto-isolator for electrical isolation of an electrical input signal from an electrical output signal comprising:a signal input terminal configured to receive the electrical input signal;an optical emitter coupled to the signal input terminal and configured to generate emitted light in response to the electrical input signal;a first optical receiver comprising a first photodetector arranged proximate to the optical emitter to receive a first portion of the emitted light, wherein the first optical receiver is configured to generate the electrical output signal in response to the emitted light;a signal output terminal coupled to the first optical receiver to receive the electrical output signal;a second optical receiver comprising a second photodetector arranged proximate to the optical emitter to receive a second portion of the emitted light, wherein the second optical receiver is configured to generate a light output electrical signal in response to the second portion of the emitted light, wherein the second optical receiver comprises:a first reference signal generator configured to generate a first reference signal; anda first comparator coupled with the first reference signal generator and the second photodetector, and configured to generate a first comparison signal in response to the first reference signal and the light output electrical signal;a correction circuitry coupled with the second optical receiver, the correction circuitry configured to make adjustments when the optical emitter degrades; anda threshold correction signal generator coupled with the first comparator and configured to generate a threshold correction signal in response the first comparison signal.
H01L	8993459	2015-03-31	A method, comprising:depositing a first portion of a first material layer on a semiconductor structure;performing a first run of a post-treatment process for modifying at least said first portion of said first material layer, wherein said post-treatment process comprises exposing said semiconductor structure to an oxidizing ambient, wherein a portion of said semiconductor structure adjacent said first material layer is oxidized during said first run of said post-treatment process;after said first run of said post-treatment process, depositing a second portion of said first material layer, said second portion being formed of substantially the same material as said first portion;after said deposition of said second portion of said first material layer, performing a second run of said post-treatment process for modifying at least said second portion of said first material layer; andforming a gate electrode of a field effect transistor above said first material layer, said first material layer being located between a channel region of said field effect transistor and said gate electrode for providing electrical insulation between said gate electrode and said channel region.
H04L	9054908	2015-06-09	A chip, comprising:a circuit to transmit one or more data bursts, the circuit comprising burst timing logic to insert gaps of a determined length between adjacent data bursts to reduce noise at a desired frequency, wherein the length of each gap is to be determined based on energy of a preceding burst and a current burst.
H01Q	8963794	2015-02-24	A loop antenna, comprising:a loop antenna resonating element formed from a sheet of conductive material that is wrapped around an axis to form a conductive loop; andan antenna feed structure that is directly fed and that is configured to indirectly feed the loop antenna resonating element.
A61F	9101457	2015-08-11	Apparatus comprising an endovascular stent-graft system, which comprises:a fenestrated stent-graft, which includes first and second end portions and a central portion disposed longitudinally therebetween, and which comprises a fenestrated support structure and a fenestrated covering element, which is securely attached to and covers at least a portion of the fenestrated support structure, wherein the fenestrated support structure and the fenestrated covering element are shaped so as to together define first and second lateral apertures in the central portion, which apertures face in generally radially opposing directions, when the fenestrated stent-graft is in a radially-expanded state thereof; anda crossing stent-graft, which includes first and second end portions and a central portion disposed longitudinally therebetween, and which comprises a crossing support structure and one or more crossing covering elements, which are securely attached to and at least partially cover the first and the second end portions, such that the central portion is at least partially uncovered when the crossing stent-graft is in a radially-expanded state thereof,wherein the fenestrated and the crossing stent-grafts are sized and shaped such that, when the crossing stent-graft is disposed through the first and the second apertures such that the central portion of the crossing stent-graft is within the central portion of the fenestrated stent-graft, the first and the second end portions of the crossing stent-graft (a) pass through the first and the second apertures, respectively, and (b) when the fenestrated and the crossing stent-grafts are in their radially-expanded states, form blood-impervious seals with the first and the second apertures, respectively, such that interior spaces defined by all of the following are in fluid communication with one another: the first and the second end portions and the central portion of the fenestrated stent-graft, and the first and the second end portions and the central portion of the crossing stent-graft, andwherein the crossing stent-graft, when in its radially-expanded state, has an hour-glass shape, and the central portion of the crossing stent-graft is shaped so as to define a stricture in the hour-glass shape.
C12N,C12P,C12Q,C12R,C12Y,G01N	9175296	2015-11-03	A strain of Chrysosporium lucknowense selected from the group consisting of W1L deposited at CBS under accession number 122189, W1L#100.1 deposited at CBS under accession number 122190, and a strain derived therefrom by gene disruption.
G02B	9217826	2015-12-22	A flexible light panel, comprising:a flexible light diffusing waveguide or a plurality of flexible light diffusing waveguides arranged to define a plurality of waveguide segments having longitudinal axes that are substantially parallel one to another, thereby forming a flexible light panel, each waveguide segment being in close proximity to an adjacent waveguide segment; and a light source or a plurality of light sources coupled to the flexible waveguides, wherein each light diffusing waveguide includes:(i) a glass core, comprising:(a) a region with a plurality of nano-sized structures within said core configured to scatter guided light via said nano-sized structures towards the outer surface, such that said light diffusing optical fiber has a scattering-induced attenuation greater than 50 dB/km at illumination wavelength; and(b) a solid glass region surrounding said region with a plurality of nano-sized structures having a higher refractive index delta than the region with a plurality of nano-sized structures; and(ii) a low index cladding surrounding said core, said cladding having a lower refractive index delta than said solid glass region.
G11C	9196379	2015-11-24	A magnetic shift register, comprising:a first supporting layer having a first surface and a plurality of trenches on the first surface extending along a first direction;a second supporting layer, filled in the plurality of trenches, wherein the first supporting layer and the second supporting layer have at least a portion substantially equal in height;a first pinning material layer, disposed between the first supporting layer and the second supporting layer, wherein end surfaces of the first pinning material layer is exposed by the first surface, wherein a height of each of the end surfaces of the first pinning material layer is higher than a height of the first surface; andat least one magnetic memory track extending along a second direction on the first surface, disposed over the first support layer, the first pinning material layer, and the second support layer, wherein the second direction is not the same as the first direction, wherein portions of the at least one magnetic memory track respectively at the end surfaces of the first pinning material layer are protrudent.
G02B	9004691	2015-04-14	A head-up display device for projecting an image on a display screen comprising:a liquid crystal display element for generating an original image;a light source for emitting light toward the liquid crystal display element from a backside of the liquid crystal display element;a reflection mirror for reflecting a light image of the original image passing through the liquid crystal display element and for projecting the light image as a projected image on the display screen; anda concave cylindrical lens having a concave surface and disposed between the liquid crystal display element and the reflection mirror,wherein the concave surface faces the reflection mirror,wherein three-dimensional coordinates are defined in the concave cylindrical lens,wherein the concave surface has a curved shape along with x axis of the three-dimensional coordinates,wherein the concave surface extends along with y axis of the three-dimensional coordinates,wherein the concave surface is configured to be rotated around the x axis so that z axis of the concave surface is tilted from an optical axis,wherein the optical axis is defined to be directed from the light source to the reflection mirror,wherein the head-up display device has a plurality of magnification ratios of the projected image with respect to the original image,wherein the concave surface of the concave cylindrical lens has a curvature radius, which corresponds to the one of the magnification ratios,wherein a view angle in the x axis is defined as a range of an angle of light, which is emitted with the angle in the x axis with respect to the optical axis from the light source as an original point toward the liquid crystal display element and has a predetermined intensity ratio with respect to straight light emitted along with the optical axis,wherein a view angle in the y axis is defined as a range of an angle of light, which is emitted with the angle in the y axis with respect to the optical axis from the light source as an original point toward the liquid crystal display element and has a predetermined intensity ratio with respect to straight light emitted along with the optical axis,wherein the view angle in the x axis is determined to be a view angle, which is necessary in the x axis for a case where the one of a plurality of magnification ratios is a minimum magnification ratio,wherein the view angle in the y axis is determined to be a view angle, which is necessary in the y axis for a case where the one of a plurality of magnification ratios is a maximum magnification ratio, andwherein the minimum magnification ratio of the view angle in the x axis corresponds to the maximum magnification ratio of the view angle in the y axis.
F21K,F21V,F21Y	8979317	2015-03-17	A light flux controlling member for controlling a distribution of light emitted from a light-emitting element, the light flux controlling member comprising:a plate-shaped first lens; anda plate-shaped second lens,wherein the first lens has a first front surface that is one principal surface, a first rear surface that is the other principal surface, and a first side surface that forms a lateral contour of the first lens,wherein the second lens has a second front surface that is one principal surface, a second rear surface that is the other principal surface, and a second side surface that forms a lateral contour of the second lens,wherein the first lens and the second lens are arranged in an overlapping manner so that the first front surface and the second rear surface face each other and that a low refractive index layer having a lower refractive index than the first lens and the second lens is present between the first front surface and the second rear surface,wherein the first lens has a first concave surface that is formed by indenting the first rear surface of the first lens and is configured to allow the light emitted from the light-emitting element to be incident on the first concave surface to generate light to be guided toward the first side surface,wherein the second lens has a second concave surface that is formed by indenting the second front surface of the second lens and is configured to emit or totally reflect the light that is incident on the first concave surface and is transmitted through the first front surface and the second rear surface, andwherein the first concave surface is inclined such that a concave space in the first lens is tapered toward the first front surface.
G06F	9176771	2015-11-03	A data processor comprising:a CPU;a plurality of peripheral devices; anda memory comprising:a plurality of device control threads for controlling operation of respective peripheral devices;a control thread including a plurality of processing parts associated with respective device control threads;a table storing configurations of a plurality of applications including application priority of respective applications, wherein each application includes an operation sequence of processing parts that constitute an application and a dependence relationship between the processing parts of the application, wherein the processing parts are prioritized such that processing part near end of the operation sequence is performed with higher priority than processing part near beginning of the operation sequence;a program, when executed by the CPU, executes the control thread to perform the method of:executing two or more of the applications sharing at least one peripheral device in parallel,wherein when an overlap between operation timings of respective processing parts of the two or more of the applications corresponding to a same peripheral device occurs, processing part with higher priority is executed before respective processing parts of other applications, whereinif priorities associated with the respective processing parts are the same, application priority of application associated with processing part that last executed is lowered such that processing part of the other application with higher priority is executed, andif each of the respective processing parts has no dependence relationship based on the configuration stored in the table, no priority is given to the respective processing parts such that the respective processing parts are executed in alternate manner.
B64C,F16F	8967534	2015-03-03	An aircraft landing gear assembly, said landing gear assembly comprised of a nonelastomeric outer aircraft landing gear vertical strut member having an inner axial center bore with a frictional interface inner surface segment, a nonelastomeric inner aircraft landing gear vertical strut member having an outer bonding surface segment, said nonelastomeric inner aircraft landing gear vertical strut member rotationally received in said outer aircraft landing gear vertical strut member inner axial center bore with said nonelastomeric inner aircraft landing gear vertical strut member rotatable within said outer aircraft landing gear vertical strut member, and an elastomeric surface effect damper member encompassing said nonelastomeric inner aircraft landing gear vertical strut member outer bonding surface segment, said elastomeric surface effect damper member having an inner bonding surface segment and an outer grooved elastomer surface, said elastomeric surface effect damper inner bonding surface segment bonded to said nonelastomeric inner aircraft landing gear vertical strut member outer bonding surface segment, and a surface effect lubricant, said surface effect lubricant disposed between said elastomeric surface effect damper outer grooved elastomer surface and said nonelastomeric outer aircraft landing gear vertical strut member frictional interface inner surface segment with said elastomeric surface effect damper outer grooved elastomer surface engaging said nonelastomeric outer aircraft landing gear vertical strut member frictional interface inner surface segment and inhibiting an oscillating shimmy rotation of said nonelastomeric inner aircraft landing gear vertical strut member.
A61B,G06F,G06T,G09G	8958611	2015-02-17	A method of transforming a cut procedure into a visual representation of a bone cut during resection via a cutting tool, wherein the cutting tool is attached to an arm of a surgical robot, the method comprising:obtaining a bone shape data collection representing a shape of a bone to be resected and generating a bone model from the bone shape data collection;generating a cutting tool model of the cutting tool;collecting tool position data representing a plurality of positions of the cutting tool relative to the bone during resection of the bone and generating a plurality of subtraction models based on the cutting tool model and the plurality of positions; andtransforming the bone model and the plurality of subtraction models into an image space visual representation of the bone cut by merging the subtraction models into a single subtraction model and subtracting the single subtraction model from the bone model, the single subtraction model representing the resection.
H01L	9129958	2015-09-08	A semiconductor package, comprising:a substrate;a top semiconductor die disposed above the substrate;an interposer having a window, the interposer disposed between and interconnected to the substrate and the top semiconductor die; anda bottom semiconductor die disposed in the window of the interposer, and interconnected to the top semiconductor die, wherein, from a top-down perspective, the top semiconductor die only partially overlaps the bottom semiconductor die, and wherein the top semiconductor die is disposed above the bottom semiconductor die, and the bottom semiconductor die is disposed above external conductive contacts of the interposer.
H04B,H04L,H04W	9055438	2015-06-09	A method comprising:transmitting a first credential from a mobile device to a proximate first device, the first device validating the first credential;receiving, at the mobile device, a second credential from the first device; andtransmitting the second credential from the mobile device to a proximate second device, the second device to, upon validating the second credential, establish a session to exchange data with the first device.
H02K	9093880	2015-07-28	A permanent magnet motor comprising:a stator core formed of a soft magnetic material and including a core portion and a plurality of chip portions each comprised of dust core portion;a stator having a coil wound on the stator core;a rotor having a permanent magnet,wherein the core portion includes a number of steel plates stacked and secured to one another in a direction of a rotation axis, the steel plates being individually insulated from each other, the core portion including an annular path portion and a plurality of salient pole portions;the chip portions are attached to distal ends of the salient pole portions respectively, each chip portion having at least one axial end axially protruding from an axial end of each salient pole portion and one or a plurality of radially extending protrusions;the salient pole portions are magnetically routed by the path portion and extend radially from the path portion;the salient pole portions have distal end sides formed with recesses capable of accommodating the protrusions, respectively; andthe protrusions of the chip portions are accommodated in the recesses of the salient pole portions, whereby the chip portions are attached to the core portion.
G06F,G09G,H04M,H04N	9019087	2015-04-28	A method comprising:receiving a media file having embedded haptic event data in a tag preceding content data in the media file, the haptic event data having time stamp information embedded in one or more frames of the haptic event data;synchronizing the embedded haptic event data with the content data in the media file by assigning one or more time stamps to the one or more frames of the haptic event data according to a master time code embedded in the media file, wherein each time stamp is synchronized with a corresponding frame of the content data, wherein each time stamp is embedded within the tag preceding the content data in the media file, wherein at least one time stamp of the one or more time stamps is a sub-time stamp that is an offset of another time stamp within a single frame of the haptic event data; andoutputting one or more haptic effects via an actuator in a playback device, wherein each outputted haptic effect is synchronized with a corresponding media event of the content data, each media event and corresponding haptic event occurring at a substantially same time stamp, and wherein at least one haptic effect is output at the offset based on an offset parameter indicated within the haptic event data.
A23C,A23L,A23V,A61K,C12N	9040036	2015-05-26	A pharmaceutical composition comprising viable non-pathogenic Clostridia spores, a viable non-pathogenic Bacteroides, and a viable non-pathogenic Escherichia coli.
G01C,G01S,G05D,G06F,Y10S	8972052	2015-03-03	A navigation control system for an autonomous vehicle, the system comprising:a transmitter comprising:a first emitter for emitting a first signal;a second emitter for emitting a second signal;a power source for powering both the first emitter and the second emitter;an antenna for capturing wireless energy; anda printed circuit board for converting the captured wireless energy to a form for charging the power source powering both the first emitter and the second emitter; andan autonomous vehicle operating within a working area, the autonomous vehicle having a receiving system comprising:a receiver for detecting the first and second signals; anda processor for determining a relative location of the autonomous vehicle within the working area based on the first and second signals, wherein a source of the wireless energy is located on the autonomous vehicle.
G01V	9128203	2015-09-08	A computer implemented method of modeling a reservoir property of subsurface reservoir structure by support vector machine processing in the computer of input data available from the reservoir to form measures of the reservoir property at regions of interest in the subsurface reservoir by regression analysis of the available input data, the method comprising the computer processing steps of:(a) receiving training input data about subsurface attributes from seismic survey data obtained from seismic surveys of the reservoir;(b) receiving training target data about formation rock characteristics from data obtained from wells in the reservoir;(c) partitioning the subsurface attributes training data and the formation rock characteristics training target data into a plurality of subsets;(d) selecting formation attribute parameters for support vector machine modeling by performing the steps of:(1) cross-validating the subsets of subsurface attributes training data each with the other subsets of the plurality of subsets for a radial based kernel function pair comprising a kernel parameter value and a penalty parameter pair value;(2) forming an error function for each of the cross-validated subsets;(3) repeating the steps of cross-validating the subsets of subsurface attributes training data and forming an error function for a plurality of different radial based kernel function pairs;(e) optimizing the selected formation attribute parameters by determining a minimum error function of the formed error functions for the plurality of different radial based kernel function pairs;(f) providing the training data, the selected formation attribute parameters, the cross-validated subsets of subsurface attributes training data, and the error functions for the plurality of radial based function kernel pairs as training inputs for support vector machine modeling;(g) performing support vector machine modeling by regression analysis to determine a minimum error function of the error functions of the provided training inputs;(h) predicting the reservoir property based on the support vector modeling of the training inputs; and(i) forming an output display of the predicted reservoir property.
H01L,Y10S	8980441	2015-03-17	An organic electroluminescent device comprising: a cathode; an anode; and a light-emitting layer between the cathode and the anode, the light-emitting layer including a compound represented by formula (3):wherein A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A30,A31,A32, A33,A34,A35, A36,A37, A38, A39, and A40 each independently represent a carbon atom or a nitrogen atom; A17, A22,A27,and A32 each represent a carbon atom having R&#x2033;; R&#x2033; represents a substituent having an Sp3 carbon atom at a bonding position thereof; a plurality of Rs each independently represent an alkyl group having 1 to 18 carbon atoms, an aryl group having 6 to 18 carbon atoms, a halogen atom, a cyano group, a nitro group, a heterocyclic group having 2 to 10 carbon atoms, a silyl group having 3 to 18 carbon atoms or a phosphoryl group having 1 to 18 carbon atoms; and m represents an integer, further wherein the ring structure comprising A37, A38, A39, and A40 comprises a substituent linked to A38 or to A40, wherein the substituent linked to A40 is an alkyl substituent having 1 to 18 carbon atoms, wherein said substituent is linked to A40 when A40 is a carbon atom, and wherein the substituent linked to A38 is selected from the group consisting of an alkyl group having 1 to 18 carbon atoms, a halogen atom, a cyano group, a nitro group, a heterocyclic group having 2 to 10 carbon atoms, a silyl group having 3 to 18 carbon atoms or a phosphoryl group having 1 to 18 carbon atoms, wherein said substituent is linked to A38 when A38 is a carbon atom.
H01L	8936984	2015-01-20	A method of manufacturing a three-dimensional (3-D) nonvolatile memory device, the method comprising:forming a stack structure by alternately forming first material layers and second material layers over a substrate;forming channel layers extending perpendicular to a surface of the substrate, the channel layers penetrating the first and the second material layers;forming a slit, exposing a portion of the surface of the substrate, by etching the first material layers and the second material layers between the channel layers; andforming an etch-stop layer on the portion of the surface of the substrate exposed through the slit by changing the portion of the surface of the substrate into an insulating material.
H05B	9119241	2015-08-25	A light source bias control system with a single converter comprising:a first light source configured to produce light of a first wavelength having an associated first headroom value on a first terminal;a second light source configured to produce light of a second wavelength having an associated second headroom value on a second terminal;one converter configured to bias the first light source and the second light source;a switch, responsive to a switch control signal, capable of establishing:a first feedback path between the first terminal and the converter; anda second feedback path between the second terminal and the converter;at least one comparator configured to:receive and compare the first headroom value to a first reference value to generate a first comparator output;receive and compare the second headroom value to a second reference value to generate a second comparator output; andcontrol logic configured to process the first comparator output and the second comparator output to generate the switch control signal.
C08G,C08K,C08L,G03F	9085696	2015-07-21	A black photosensitive resin composition, which includes:(A) 0.5-5.0 wt % of a polysiloxane;(B) 20-70 wt % of a black pigment;(C) 1.0-30 wt % of an alkali-soluble resin;(D) 0.5-10 wt % of a photopolymerizable compound;(E) 0.5-5.0 wt % of a photopolymerization initiator; and(F) a solvent as the remainder;wherein the polysiloxane is a copolymer having a siloxane backbone, and the copolymer having a siloxane backbone includes the following formula (I) unit and formula (II) unit:wherein R1, R2 and R3 are each independently H or C1-6 alkyl group; s is an integer of 0 to 5; and r is an integer of 1 to 6.
B60W,G05B	9026349	2015-05-05	A method for modifying performance of a mining haul truck, the method comprising:determining a performance target for the mining haul truck;comparing the performance target to an actual operating parameter of the mining haul truck;modifying an actual performance of the mining haul truck to achieve the performance target;obtaining information from at least one prior haul cycle of the mining haul truck based on learning the at least one prior haul cycle of the mining haul truck;determining at least one parameter for at least one current haul cycle based on the information from the at least one prior haul cycle that is obtained; andcontrolling the mining haul truck during the at least one current haul cycle using the at least one parameter.
G06F,G11C,H01L	9189442	2015-11-17	An apparatus, comprising:NAND flash memory;random access memory;additional memory;a first circuit for receiving DDR signals and outputting SATA signals, the first circuit communicatively coupled to the additional memory, and further capable of being communicatively coupled to a first bus associated with a DDR protocol including at least one of a DDR2 protocol, a DDR3 protocol, or a DDR4 protocol; anda second circuit for receiving the SATA signals and outputting NAND flash signals, the second circuit communicatively coupled to the first circuit via a second bus associated with a SATA protocol, the second circuit further communicatively coupled to the NAND flash memory via a third bus associated with a NAND flash protocol, the second circuit further communicatively coupled to the random access memory;said apparatus configured such that:said first circuit:receives a first command including first data via the first bus for causing at least a portion of a random access data read,stores at least a portion of the first command in the additional memory,generates, in response to the first command, a second command including second data that is based on at least a portion of the first data, andsends the second command to the second circuit via the second bus;said second circuit:receives the second command from the first circuit via the second bus,generates, in response to the second command, a third command including third data that is based on at least a portion of the second data, andsends the third command to the NAND flash memory via the third bus for causing first information in the NAND flash memory to be written to the random access memory;said first circuit:receives a status query via the first bus for checking a status in connection with the first information, andallows, in response to the status query, the status to be checked in connection with the first information;said first circuit:receives a fourth command including fourth data via the first bus for causing the first information to be read from the random access memory, andsends, in response to the fourth command, the first information from the random access memory to a processor via the first bus.
C07D	9139524	2015-09-22	A compound of formula (I)whereinn represents 1 or 2; andR1 represents hydrogen or fluoro;or a salt thereof.
G06F,H04L	9112811	2015-08-18	A network system for connecting a plurality of machines that logically connect through a logical switching element, the system comprising:a first network comprising a plurality of unmanaged switching elements, at least one of which connects to a first machine that communicates with other machines of the plurality of machines through the logical switching element;a second network comprising a plurality of managed switching elements that collectively implement the logical switching element and communicate with each other via tunnels provided by tunneling protocols, wherein a first managed switching element that implements the logical switching element connects to a second machine that communicates through the logical switching element, and a second managed switching element connects to a third machine that also communicates through the logical switching element; anda network extending managed switching element located at an edge of the first network for facilitating implementation of the logical switching element that logically connects the first, second, and third machines between the first network and the second network, wherein the network extending managed switching element adds logical context tags to packets sent by the first machine and received from the at least one unmanaged switching element before forwarding said packets to the second network and removes logical context tags from packets received from the second network before forwarding said packets to the at least one unmanaged switching element and the first machine.
G06F	9002888	2015-04-07	A method of minimizing epigenetic surprisal data comprising:a computer receiving a request for an epigenetic time series analysis comprising at least an indication of a baseline epigenetic surprisal data and a time period for the epigenetic time series analysis;the computer retrieving a baseline epigenetic surprisal data and epigenetic surprisal data for at least one time point within the time period from a repository; andthe computer comparing the at least one epigenetic surprisal data comprising a location of an epigenetic modification, a time in which the epigenetic modification was collected and a type of epigenetic modification for the at least one time point within the time period to the baseline epigenetic surprisal data comprising a location of an epigenetic modification, a time in which the epigenetic modification was collected and a type of epigenetic modification, producing at least one epigenetic modification difference;the computer storing the at least one epigenetic modification difference in a repository; andrepeating the method for all epigenetic surprisal data time points within the time period.
F16B	9028185	2015-05-12	An interconnectable pair of an insert and a first part, the interconnectable pair being adapted to be joined to a second part, said interconnectable pair comprising:an insert with a longitudinal body, an outer surface of said longitudinal body including male threads;an annular flange on one end of said longitudinal body, wherein said annular flange has an inner and outer circumference, the inner circumference defined by an intersection between said longitudinal body and said annular flange;a hole in a center of said longitudinal body, said hole including female threads;protrusions and recesses on said annular flange, wherein said protrusions and recesses extend inward along an axial direction of the annular flange, between the inner circumference and the outer circumference of said annular flange;wherein said protrusions and recesses alternate in a radial pattern around the annular flange; andan opening in said first part, said opening comprising a hole and a counter bore hole;said counter bore hole including recesses and protrusions molded therein, said recesses and protrusions alternating in a radial pattern around the counter bore hole and said recesses and protrusions molded in the counter bore hole being present within the counter bore hole prior to insertion of the insert into the opening.
G01N,G01S	9176056	2015-11-03	A method of measuring the reflectivity of a region of an article to electromagnetic radiation comprising the steps of:a) arranging the article in the near-field of a transceiver antenna,b) aligning the antenna along a direction substantially orthogonal to a surface of the region to be measured and at a predetermined distance from said surface,c) illuminating the region of the article with electromagnetic radiation transmitted from the antenna over a desired frequency range,d) receiving electromagnetic radiation reflected from the region of the article in the near-field of the antenna using said antenna,e) measuring the reflected radiation and determining there-from the reflectivity of said article in said region;wherein at least one substantially rigid spacing member is provided in mechanical communication with the article and the antenna so as to align the antenna, andthe spacing member comprises an optical element comprising a polymer foam disposed between the antenna and the article and wherein the illuminating and measuring steps are performed through said optical element.
G06K,G06N,G10L	9117444	2015-08-25	A method of classifying input data representing speech input by a user of a speech application as belonging to one of a plurality of classifications, the plurality of classifications associated with a respective plurality of clusters that were fit to training data representing a plurality of speech utterances associated with the speech application, the method comprising:using at least one processor to perform:obtaining a first transformation generated based, at least in part, on a plurality of representative vectors determined from the training data, the plurality of representative vectors including at least one representative vector determined for each of the plurality of classifications, wherein the plurality of clusters were fit to the training data at least in part by fitting the plurality of clusters to transformed training data obtained by applying the first transformation to the training data;transforming the input data using at least the first transformation to obtain transformed input data representing the speech input by the user of the speech application;comparing the transformed input data to the plurality of clusters to determine which cluster of the plurality of clusters the input data should be associated with; andclassifying the input data according to a classification of the plurality of classifications associated with the determined cluster.
H01G,H01L	9006074	2015-04-14	A method of forming an integrated circuit, comprising the steps of:providing a substrate comprising a semiconductor;forming a plurality of transistors in said substrate;forming at least one metal level over said substrate, said metal level containing metal interconnects;forming a bottom plate of an isolation capacitor;forming a silicon dioxide dielectric layer of said isolation capacitor over said bottom plate;forming a polymer dielectric layer of said isolation capacitor over said silicon dioxide dielectric layer, said polymer dielectric layer extending across said integrated circuit;forming a via hole through said polymer dielectric layer;forming a top plate of said isolation capacitor over said polymer dielectric layer;forming a bond pad on said top plate; andforming another bond pad outside said isolation capacitor, said another bond pad being electrically coupled to an instance of said metal interconnects through a via in said via hole.
E21B	9010420	2015-04-21	A sonic oil recovery apparatus for use in a well having a casing therein, the apparatus comprising:an injector tubing suitably extending interior of the casing; anda resonator tube affixed to or within said injector tubing, said resonator tube having an interior flow pathway so as to allow a fluid to flow therethrough from said injector tubing, said resonator tube suitable for transmitting an acoustic signal of greater than 30 Hz therefrom, said resonator tube having a plurality of orifice plates formed therein.
B64D,F02C,F02K,F05D,Y02T	8925332	2015-01-06	A seal assembly for a turbojet nacelle, comprising:a first end equipped with means for fastening to at least a first structure of the turbojet nacelle;a second end configured to come into contact against a bearing zone of a second structure of the turbojet nacelle; andat least two rows of blades each comprising a plurality of adjacent blades arranged along the first end and extending substantially longitudinally and substantially perpendicularly thereto,wherein at least one of the plurality of adjacent blades has an accordion-shaped structure, wherein said at least two rows of blades are radially spaced apart from each other with respect to a longitudinal axis of the adjacent blades, wherein said two rows of blades are circumferentially distributed around said longitudinal axis, and wherein said two rows of blades are configured to allow compression and release of the accordion-shaped structure without interfering with an adjacent row.
A61B	8956285	2015-02-17	A method of enlarging an incision, the method comprising:inserting a first pair of opposing tissue engaging members into the incision;connecting the first pair of tissue engaging members to initial stage operating mechanisms of a retractor;retracting the first pair of tissue engaging members away from each other to enlarge the incision to form an intermediate size opening;positioning mounting portions of subsequent stage operating mechanisms of the retractor above the intermediate size opening after retracting the first pair of tissue engaging members to form the intermediate size opening;inserting a second pair of opposing tissue engaging members in a downward direction into the intermediate size opening after positioning the mounting portions of the subsequent stage operating mechanisms above the intermediate size opening; andsliding attachment portions of the second pair of tissue engaging members in the downward direction onto the mounting portions of the subsequent stage operating mechanisms as the second pair of tissue engaging members are inserted into the intermediate size opening after positioning the mounting portions of the subsequent stage operating mechanisms above the intermediate size opening to connect the second pair of tissue engaging members to the subsequent stage operating mechanisms.
A61B	9186197	2015-11-17	A cryoablation device comprising: a probe having an exterior and an interior, the exterior comprising a tip and the interior defining a passage having a filament and insulation disposed therein, the filament configured to conduct a pressurized material at a temperature from &#x2212;180&#xb0; C. to about 0&#xb0; C. and having an opening configured to release the pressurized material into the interior of the probe so as to cool the tip to a selected temperature to ablate nerve and/or soft tissue; and a bone piercing instrument, the bone piercing instrument configured to pierce bone so as to allow entry of the probe into the bone, and a pressure of the pressurized material is from about 3,000 to about 6,000 pounds per square inch (PSI).
F16B	8973877	2015-03-10	A sucker comprising:a sucker body; anda driving mechanism connected with the sucker body; wherein:the sucker body includes:a rough suction layer;a support layer combined with the rough suction layer;a receiving chamber connected with the support layer;a mounting seat connected with the support layer; andan airtight control switch pivotally connected with the mounting seat;the rough suction layer of the sucker body is made of soft material;the support layer of the sucker body is made of hard material;the rough suction layer of the sucker body includes a plurality of protrusions juxtaposed to each other closely and a plurality of grooves defined between the plurality of protrusions;the rough suction layer of the sucker body has a central portion provided with an air recess which has an air channel;the receiving chamber of the sucker body is connected to the air channel of the rough suction layer;the mounting seat of the sucker body has a bottom provided with an air hole;the support layer of the sucker body has a side provided with a conduit connected between the air channel of the rough suction layer and the air hole of the mounting seat;when the rough suction layer of the sucker body presses an attached face and is disposed at a negative pressure state, the sucker body is pressed toward the attached face by a normal force from ambient air, so that the rough suction layer of the sucker body is compressed, and the plurality of grooves of the rough suction layer form multiple independent negative pressure or vacuum zones, to increase a negative pressure of the rough suction layer, to decrease air existing in the rough suction layer of the sucker body, and to reduce a vacuum leak in the rough suction layer of the sucker body, to increase an air-tightness between the rough suction layer of the sucker body and the attached face; andthe rough suction layer of the sucker body is bonded onto the attached face closely and tightly so that when the attached face is an unsmooth or cracked face, the rough suction layer of the sucker body will not be detached from the attached face.
A61B	9008269	2015-04-14	A method for automatically adjusting a field of view for exposure of an X-ray system, the method comprising:capturing an image of a patient on an examining table of the X-ray system by an image sensor, wherein the image sensor is placed at a predetermined position in the X-ray system;displaying the captured image on a display for selection of a region of interest or a point of interest by a user on the image;automatically determining a target position of an X-ray source in response to the selection of the region of interest or the point of interest on the image by at least:determining a position of a center of a desired field of view for exposure on the image in response to the selection of the region of interest or the point of interest; anddetermining the target position of the X-ray source based on the position of the center of the desired field of view for exposure on the image and imaging parameters of the image sensor by at least:determining an actual position of the center of the desired field of view for exposure on the patient based on the position of the center of the desired field of view for exposure on the image and the imaging parameters of the image sensor; anddetermining the target position of the X-ray source based on the actual position of the center of the desired field of view for exposure on the patient and inherent positional relationship between the X-ray source and the center,wherein the desired field of view for exposure covering the region of interest or the point of interest is obtained when the X-ray source is located at the target position; andautomatically locating the X-ray source at the target position in response to the determination of the target position.
G06F,H04L,Y02B	9069611	2015-06-30	A method comprising:determining, at a current time, an availability of compute resources within a compute environment for each workload in a queue of workload based on scheduled times of compute resources established for the each workload to yield an analysis, the each workload to consume scheduled compute resources at a future time, which is later than the current time, according to the scheduled times;based on the analysis, determining an ability of the compute environment to satisfy a service level agreement associated with the each workload in the queue of workload based on a current configuration of the compute environment;determining possible workload scheduling changes in the compute environment to improve service levels for at least one workload in the queue of workload;determining a cost of provisioning compute resources associated with the possible workload scheduling changes; andif the possible workload scheduling changes improve service levels within the compute environment, taking into account the cost, rescheduling the at least one workload in the compute environment prior to all workload in the queue of workload consuming compute resources in the compute environment.
B23K,F16D	9193009	2015-11-24	A clutch having a cover plate (26), a pre-clutch (1) and a main clutch (2), where the pre-clutch (1) can be brought to an engaged or a disengaged state by a tensioning means (4), the pre-clutch (1) being connected to the main clutch (2) by means of an operative connection (80) in such a way that the disengaged position and the engaged position of the pre-clutch (1) are transmitted to the main clutch (2), wherein:the tensioning means (4) is biased between said cover plate (26) and a driver disk (7) of a clutch basket (18);the operative connection (80) has a ramp (12), a transmitting element (13) and a pressure plate (14);the ramp (12) is coupled with the pre-clutch (1);the pressure plate (14) is in an operative connection with the main clutch (2);the transmitting element (13) is situated between the ramp (12) and the pressure plate (14) to transmit power;the pressure plate (14) is directly connected to a clutch core (11) by spring means (82) connected to a bolt (39) of clutch core (11);said ramp is supported by a needle bearing (10); and,said spring means (82) is a leaf spring.

Writing test_data.tsv


In [ ]:
src_folder = '/content/patent.bert'
if not os.path.exists(src_folder):
  !mkdir $src_folder
  !cp *.py $src_folder

print('prepared source code:')
!ls $src_folder

print('prepared test data:')
!cp *.tsv $model_folder
!ls $model_folder/test_data.tsv

prepared source code:
modeling.py	 patent_classifier.py  tokenization.py
optimization.py  __pycache__
prepared test data:
/content/PatentBERT/test_data.tsv


In [ ]:
# Run patent_classifier.py

TEST_FILE = 'test_data.tsv'  # or 'data.2015.tsv'
pred_result_file = 'predict_result.txt'
THRESHOLD = 0.3

BERT_MODEL = 'uncased_L-12_H-768_A-12' 
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
SRC_VOCAB = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
SRC_CONFIG = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
BERT_FINETUNED_DIR = '/content/PatentBERT'
VOCAB_FILE = os.path.join(BERT_FINETUNED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_FINETUNED_DIR, 'bert_config.json')
if os.path.exists(VOCAB_FILE) == False:
  !gsutil cp $SRC_VOCAB $BERT_FINETUNED_DIR
if os.path.exists(CONFIG_FILE) == False:
  !gsutil cp $SRC_CONFIG $BERT_FINETUNED_DIR

INIT_CHECKPOINT = os.path.join(BERT_FINETUNED_DIR, 'model.ckpt-181172')
label_file = os.path.join(BERT_FINETUNED_DIR, 'labels_group_id.tsv')
PREDICT_BATCH_SIZE = 8
number_of_predictions = -1 # -1 for all

program = 'patent_classifier.py'   # release version
command = "python patent.bert/" + program + " \
  --multi_hot_threshold=" + str(THRESHOLD) + " \
  --test_file=" + TEST_FILE + " \
  --predict_batch_size=" + str(PREDICT_BATCH_SIZE) + " \
  --vocab_file=" + VOCAB_FILE + " \
  --bert_config_file=" + CONFIG_FILE + "  \
  --init_checkpoint=" + INIT_CHECKPOINT + " \
  --pred_result_file=" + str(pred_result_file) + " \
  --number_of_predictions=" + str(number_of_predictions) + " \
  --label_file=" + str(label_file) + " \
  --data_dir=" + str(model_folder) + " \
  --output_dir=" + str(model_folder) + " \
  --do_lower_case=" + str(DO_LOWER_CASE) + " "

print('command: %s' % command)  
!{command}

print('\nPrediction results:')
!head -n 100 $pred_result_file

Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt...
/ [1 files][226.1 KiB/226.1 KiB]                                                
Operation completed over 1 objects/226.1 KiB.                                    
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json...
/ [1 files][  313.0 B/  313.0 B]                                                
Operation completed over 1 objects/313.0 B.                                      
command: python patent.bert/patent_classifier.py   --multi_hot_threshold=0.3   --test_file=test_data.tsv   --predict_batch_size=8   --vocab_file=/content/PatentBERT/vocab.txt   --bert_config_file=/content/PatentBERT/bert_config.json    --init_checkpoint=/content/PatentBERT/model.ckpt-181172   --pred_result_file=predict_result.txt   --number_of_predictions=-1   --label_file=/content/PatentBERT/labels_group_id.tsv   --data_dir=/content/PatentBERT   --output_dir=/content/PatentBERT   --do_lower_case=True 

/usr